# VAE + KMeans + Boundary Clusters Detection

## Цель
Целью настоящего исследования является разработка гибридного алгоритма кластеризации на основе комбинации Variational Autoencoder и метода K-Means с механизмом обнаружения промежуточных кластеров  предоставляющих дополнительную информацию о переходных кластерах между основными кластерами для практического применения.

**Промежуточные кластеры** - это плотные кластеры между основными кластерами, которые пересекаются. Для бизнеса это дополнительная важная информация о пользователях, которых нельзя однозначно отнести к какой-либо группе


---

## Научная новизна

### 1: Интегральная boundary-метрика
Комбинирует:
- **Соотношение расстояний**: $r = \frac{d_1}{d_2}$ (расстояния до двух ближайших центров)
- **Энтропию соседей**: $H = -\sum_{i} p_i \log p_i$ (разнообразие меток среди k соседей)
- **Комбинированная метрика**: $score = 0.9\cdot r_{norm} + 0.1 \cdot H_{norm}$

### 2: Схема группировки boundary-точек
1. Группировка по парам ближайших основных кластеров $(c_i, c_j)$
2. DBSCAN при превышении лимита кластеров

### 3: Boundary-метрики в оптимизации
Использование boundary-метрик как целевого критерия при оптимизации гиперпараметров.


In [ ]:
# @title
# Установка зависимостей и импорт библиотек
import sys
import subprocess
import os

# Функция установки зависимостей из requirements.txt
def install_from_requirements():
    """
    Устанавливает зависимости из requirements.txt
    """
    current_dir = os.getcwd()
    requirements_path = os.path.join(current_dir, "requirements.txt")

    if not os.path.exists(requirements_path):
        parent_dir = os.path.dirname(current_dir)
        requirements_path = os.path.join(parent_dir, "requirements.txt")

        if not os.path.exists(requirements_path):
            return install_manually()

    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", "-r", requirements_path],
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            return True
        else:
            return install_manually()
    except Exception:
        return install_manually()

# Функция установки зависимостей вручную (fallback)
def install_manually():
    """
    Устанавливает зависимости вручную (fallback)
    """
    packages = [
        "pandas>=1.5.0",
        "numpy>=1.21.0",
        "matplotlib>=3.5.0",
        "seaborn>=0.12.0",
        "scikit-learn>=1.0.0",
        "scipy>=1.9.0",
        "torch>=1.12.0",
        "torchvision>=0.13.0",
        "scikit-optimize>=0.9.0",
        "statsmodels>=0.13.0",
        "scikit-fuzzy>=0.4.2",
        "plotly>=5.0.0"
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        except Exception:
            pass

    return True

# Проверяем наличие основных библиотек и устанавливаем при необходимости
try:
    import pandas
    import numpy
    import matplotlib
    import seaborn
    import sklearn
    import scipy
    import torch
except ImportError:
    install_from_requirements()

# Устанавливаем зависимости (если нужно)
install_from_requirements()

# Добавляем текущую директорию в путь для импорта imports.py
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Импортируем все из модуля imports
from imports import *


## Общая методология подхода

Предлагаемый алгоритм кластеризации состоит из трех основных этапов:

### 1. Снижение размерности через VAE

**Зачем VAE?**
- VAE учится находить скрытые структуры в данных
- Латентное пространство VAE лучше подходит для кластеризации, чем исходное пространство признаков
- VAE автоматически извлекает наиболее важные признаки и их комбинации
- Регуляризация через KL divergence обеспечивает гладкое и интерпретируемое латентное пространство

### 2. Кластеризация на латентном пространстве

**KMeans на латентных представлениях**:
- Работает эффективнее, чем на исходных данных
- Находит основные, хорошо разделенные кластеры
- Использует среднее значение латентного распределения (μ) для детерминированности

### 3. Обнаружение промежуточных (boundary) кластеров

**Научная новизна**: Выделение промежуточных кластеров между основными кластерами:
- Использует интегральную метрику (расстояния + энтропия)
- Группирует boundary точки в отдельные кластеры
- Предоставляет дополнительную информацию для бизнеса о "неопределенных" пользователях


## Предобработка данных


In [ ]:
# @title
def preprocess_dataframe(df, exclude_columns=None):
    """
    Универсальная функция предобработки DataFrame.

    Автоматически определяет числовые и категориальные признаки и выполняет:
    - Стандартизацию числовых признаков
    - One-hot encoding категориальных признаков
    - MinMax нормализацию для VAE (диапазон [0, 1])

    Параметры:
    -----------
    df : pd.DataFrame
        Входной DataFrame
    exclude_columns : list, optional
        Список колонок для исключения из обработки

    Возвращает:
    -----------
    X : np.ndarray
        Предобработанные данные (shape: [n_samples, n_features])
    preprocessor : ColumnTransformer
        Обученный препроцессор для будущего использования
    feature_info : dict
        Информация о признаках (числовые, категориальные)
    """

    # Определяем колонки для исключения
    exclude = set()
    if exclude_columns:
        exclude.update(exclude_columns)

    # Автоматическое определение типов признаков
    numeric_features = []
    categorical_features = []

    for col in df.columns:
        if col in exclude:
            continue

        # Числовые признаки: int или float типы
        if df[col].dtype in ['int64', 'int32', 'float64', 'float32']:
            # Проверяем, что это действительно числовой признак (не ID)
            # Если уникальных значений меньше 20% от размера - считаем категориальным
            # Для маленьких датасетов: если уникальных значений <= 10, считаем категориальным
            # Для больших: если < 20% уникальных значений и < 50 уникальных
            nunique = df[col].nunique()
            n_samples = len(df)

            # Проверка на бинарные признаки
            unique_values = set(df[col].dropna().unique())
            is_binary = (nunique == 2 and unique_values.issubset({0, 1, True, False, 0.0, 1.0}))

            if is_binary:
                # Бинарный признак - оставляем как числовой
                numeric_features.append(col)
            elif n_samples <= 20:
                # Маленький датасет: более строгая проверка
                if nunique <= 10 and nunique < n_samples:
                    categorical_features.append(col)
                else:
                    numeric_features.append(col)
            else:
                # Большой датасет: стандартная логика
                if nunique < n_samples * 0.2 and nunique < 50:
                    categorical_features.append(col)
                else:
                    numeric_features.append(col)
        else:
            # Все остальное - категориальное
            categorical_features.append(col)

    print(f"Найдено признаков:")
    print(f"   - Числовых: {len(numeric_features)}")
    print(f"   - Категориальных: {len(categorical_features)}")

    # Проверка: все ли колонки обработаны
    processed_cols = set(numeric_features + categorical_features)
    all_cols = set(df.columns) - exclude
    unprocessed = all_cols - processed_cols
    if unprocessed:
        print(f"Следующие колонки не обработаны: {list(unprocessed)}")
        print("   Они будут исключены из анализа!")

    if len(numeric_features) == 0 and len(categorical_features) == 0:
        raise ValueError("Не найдено признаков для обработки!")

    # Обрабатываем NaN в числовых признаках ДО нормализации
    # Заполняем NaN медианным значением для каждого признака (более устойчиво к выбросам, чем среднее)
    for col in numeric_features:
        if df[col].isna().any():
            n_missing = df[col].isna().sum()
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)
            print(f"   Заполнено {n_missing} пропущенных значений в '{col}' медианным значением: {median_value:.4f}")

    # Создание препроцессора
    transformers = []
    n_categories_before = {}  # Инициализация для случая, когда нет категориальных признаков

    # Используем только MinMaxScaler для числовых признаков
    # MinMaxScaler нормализует данные в диапазон [0, 1] для VAE
    if numeric_features:
        transformers.append(("num", MinMaxScaler(feature_range=(0, 1)), numeric_features))

    if categorical_features:
        # Обрабатываем NaN в категориальных признаках ДО преобразования в строки
        # Это обеспечивает последовательную обработку пропущенных значений
        for col in categorical_features:
            # Заполняем NaN явным значением "_MISSING_" для создания отдельной категории
            df[col] = df[col].fillna("_MISSING_").astype(str)

        # Сохраняем количество категорий ДО преобразования в строки (после обработки NaN)
        n_categories_before = {col: df[col].nunique() for col in categorical_features}

        transformers.append(("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features))

    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='drop',  # Колонки, не попавшие в transformers, будут исключены
    )

    # Применение предобработки
    X = preprocessor.fit_transform(df).astype(np.float32)

    # Проверка: сколько признаков получилось после OneHot encoding
    n_features_after_onehot = X.shape[1]
    n_numeric = len(numeric_features)
    n_categorical = len(categorical_features)
    if n_categorical > 0:
        # Подсчитываем количество категорий (используем сохраненные значения)
        n_categories = sum(n_categories_before.values())
        expected_features = n_numeric + n_categories
        if abs(n_features_after_onehot - expected_features) > 0:
            print(f"Внимание: После OneHot encoding получено {n_features_after_onehot} признаков")
            print(f"   Ожидалось: {n_numeric} числовых + {n_categories} категориальных = {expected_features} признаков")
        else:
            print(f"После OneHot encoding: {n_numeric} числовых + {n_categories} категориальных = {n_features_after_onehot} признаков")

    # Обработка NaN и бесконечных значений
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    X = np.clip(X, 0.0, 1.0)  # Гарантируем диапазон [0, 1] (защита от возможных выбросов)

    feature_info = {
        'numeric_features': numeric_features,
        'categorical_features': categorical_features,
        'excluded_columns': list(exclude)
    }

    print(f"Предобработка завершена")
    print(f"   - Размерность данных: {X.shape}")
    print(f"   - Диапазон значений: [{X.min():.4f}, {X.max():.4f}]")

    return X, preprocessor, feature_info

print("Функция предобработки определена")

Функция предобработки определена


## VAE (Variational Autoencoder) модель

**VAE** - это генеративная модель, которая учится сжимать данные в латентное пространство меньшей размерности.

### Математика VAE:

**Loss функция состоит из двух частей:**
1. **Reconstruction Loss** (ошибка восстановления): $L_{recon} = \sum_{i} BCE(x_i, \hat{x}_i)$
   - Измеряет, насколько хорошо модель восстанавливает исходные данные
   
2. **KL Divergence** (регуляризация): $L_{KL} = -0.5 \sum_{j} (1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2)$
   - Заставляет латентное пространство следовать стандартному нормальному распределению
   - Это обеспечивает гладкое и интерпретируемое латентное пространство

**Общий loss**: $L_{total} = L_{recon} + \beta \cdot L_{KL}$

Где $\beta$ - гиперпараметр, контролирующий баланс между точностью восстановления и регулярзацией.


## Методология обучения VAE

### 1. Защита от коллапса латентного пространства

**Проблема**: При обучении VAE может произойти "коллапс" латентного пространства, когда KL divergence становится очень маленьким, и модель игнорирует регуляризацию.

**Решение**:
- Динамическое увеличение коэффициента β при обнаружении малого KL loss
- Ограничение минимального значения KL loss для предотвращения полного коллапса
- Мониторинг соотношения KL loss к Reconstruction loss

### 2. Оптимизация гиперпараметров

- **Early Stopping**: Остановка обучения при отсутствии улучшения loss
- **Learning Rate Scheduling**: Автоматическое уменьшение learning rate при плато
- **Gradient Clipping**: Предотвращение взрывающихся градиентов

### 3. Использование латентного пространства

После обучения VAE мы используем **среднее значение** латентного распределения (μ) для кластеризации, а не случайные выборки. Это обеспечивает:
- Детерминированность результатов
- Более стабильные и интерпретируемые представления
- Лучшую разделимость кластеров


In [ ]:
# @title
class VAE(nn.Module):
    """
    Variational Autoencoder с настраиваемой архитектурой.

    Архитектура:
    - Encoder: входные данные → скрытые слои → (μ, log σ²)
    - Reparameterization Trick: z = μ + ε·σ, где ε ~ N(0,1)
    - Decoder: z → скрытые слои → восстановленные данные
    """

    def __init__(self, input_dim, hidden_dims, latent_dim, dropout=0.2, use_batch_norm=True):
        """
        Параметры:
        -----------
        input_dim : int
            Размерность входных данных
        hidden_dims : list
            Список размерностей скрытых слоев (например, [512, 256, 128])
        latent_dim : int
            Размерность латентного пространства
        dropout : float
            Вероятность dropout для регуляризации
        use_batch_norm : bool
            Использовать ли Batch Normalization
        """
        super(VAE, self).__init__()

        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # ========== ENCODER ==========
        # Сжимает данные в латентное представление
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(nn.Linear(prev_dim, hidden_dim))
            if use_batch_norm:
                encoder_layers.append(nn.BatchNorm1d(hidden_dim))  # Стабилизация обучения
            encoder_layers.append(nn.ReLU())  # Активация
            if dropout > 0:
                encoder_layers.append(nn.Dropout(dropout))  # Регуляризация
            prev_dim = hidden_dim

        self.encoder = nn.Sequential(*encoder_layers)

        # Выходы encoder: μ (среднее) и log σ² (логарифм дисперсии)
        # Это параметры нормального распределения в латентном пространстве
        self.fc_mu = nn.Linear(prev_dim, latent_dim)      # Среднее μ
        self.fc_logvar = nn.Linear(prev_dim, latent_dim)  # Логарифм дисперсии log σ²

        # ========== DECODER ==========
        # Восстанавливает данные из латентного представления
        decoder_layers = []
        prev_dim = latent_dim
        # Обратный порядок скрытых слоев
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.append(nn.Linear(prev_dim, hidden_dim))
            if use_batch_norm:
                decoder_layers.append(nn.BatchNorm1d(hidden_dim))
            decoder_layers.append(nn.ReLU())
            if dropout > 0:
                decoder_layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim

        # Финальный слой с Sigmoid (для данных в диапазоне [0, 1])
        decoder_layers.append(nn.Linear(prev_dim, input_dim))
        decoder_layers.append(nn.Sigmoid())

        self.decoder = nn.Sequential(*decoder_layers)

    def encode(self, x):
        """
        Кодирует входные данные в параметры распределения.

        Возвращает:
        -----------
        mu : torch.Tensor
            Среднее латентного распределения
        logvar : torch.Tensor
            Логарифм дисперсии латентного распределения
        """
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        """
        Reparameterization Trick: z = μ + ε·σ

        Позволяет дифференцировать через случайную выборку.
        Вместо прямой выборки из N(μ, σ²) мы:
        1. Выбираем ε ~ N(0, 1)
        2. Вычисляем z = μ + ε·exp(0.5·log σ²)

        Это делает процесс дифференцируемым для backpropagation.
        """
        std = torch.exp(0.5 * logvar)  # σ = exp(0.5·log σ²)
        eps = torch.randn_like(std)     # ε ~ N(0, 1)
        return mu + eps * std           # z = μ + ε·σ

    def decode(self, z):
        """Декодирует латентное представление обратно в данные."""
        return self.decoder(z)

    def forward(self, x):
        """
        Прямой проход через VAE.

        Возвращает:
        -----------
        recon_x : torch.Tensor
            Восстановленные данные
        mu : torch.Tensor
            Среднее латентного распределения
        logvar : torch.Tensor
            Логарифм дисперсии
        """
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar


def vae_loss(x, recon_x, mu, logvar, beta=1.0):
    """
    Вычисляет loss для VAE.

    Формула: L = L_recon + β·L_KL

    Где:
    - L_recon = BCE(x, x̂) - ошибка восстановления
    - L_KL = -0.5·Σ(1 + log(σ²) - μ² - σ²) - KL divergence для регуляризации
    - β - коэффициент, контролирующий баланс

    Параметры:
    -----------
    x : torch.Tensor
        Исходные данные
    recon_x : torch.Tensor
        Восстановленные данные
    mu : torch.Tensor
        Среднее латентного распределения
    logvar : torch.Tensor
        Логарифм дисперсии
    beta : float
        Коэффициент для KL divergence (β-VAE)

    Возвращает:
    -----------
    total_loss : torch.Tensor
        Общий loss
    recon_loss : torch.Tensor
        Loss восстановления
    kl_loss : torch.Tensor
        KL divergence loss
    """
    # Reconstruction Loss
    recon_loss = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')

    logvar_clamped = torch.clamp(logvar, min=-5.0, max=5.0)
    kl_loss = -0.5 * torch.sum(1 + logvar_clamped - mu.pow(2) - logvar_clamped.exp())
    kl_loss = torch.clamp(kl_loss, min=min_kl_value)

    # Общий loss
    total_loss = recon_loss + beta * kl_loss

    return total_loss, recon_loss, kl_loss


def train_vae(X, hidden_dims=[512, 256, 128], latent_dim=15, epochs=200,
              batch_size=256, lr=1e-4, beta=4.0, dropout=0.2,
              use_batch_norm=True, verbose=True, early_stopping_patience=20):
    """
    Обучает VAE модель.

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные (должны быть в диапазоне [0, 1])
    hidden_dims : list
        Архитектура скрытых слоев
    latent_dim : int
        Размерность латентного пространства
    epochs : int
        Количество эпох обучения
    batch_size : int
        Размер батча
    lr : float
        Learning rate
    beta : float
        Коэффициент для KL divergence (β-VAE)
    dropout : float
        Вероятность dropout
    use_batch_norm : bool
        Использовать Batch Normalization
    verbose : bool
        Выводить ли прогресс обучения
    early_stopping_patience : int
        Количество эпох без улучшения для early stopping

    Возвращает:
    -----------
    latent_vectors : np.ndarray
        Латентные представления всех данных (используется μ, среднее)
    model : VAE
        Обученная модель
    history : dict
        История обучения (loss, recon_loss, kl_loss)
    """
    # Создание DataLoader
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Инициализация модели
    model = VAE(
        input_dim=X.shape[1],
        hidden_dims=hidden_dims,
        latent_dim=latent_dim,
        dropout=dropout,
        use_batch_norm=use_batch_norm
    ).to(DEVICE)

    # Оптимизатор и scheduler
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, min_lr=1e-6
    )

    # Early stopping
    best_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    history = {'loss': [], 'recon_loss': [], 'kl_loss': []}

    # Переменная для динамического beta
    current_beta = beta
    kl_threshold = 0.1  # Порог для обнаружения коллапса
    min_kl_target = 0.5  # Минимальная целевая величина KL loss
    critical_kl_threshold = 0.05
    max_beta_multiplier = 50.0

    # Обучение
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        epoch_recon = 0.0
        epoch_kl = 0.0

        for (batch_X,) in loader:
            batch_X = batch_X.to(DEVICE)

            # Прямой проход
            recon, mu, logvar = model(batch_X)
            # Используем динамический beta для защиты от коллапса
            total_loss, recon_loss, kl_loss = vae_loss(batch_X, recon, mu, logvar, beta=current_beta)

            # Обратный проход
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()

            epoch_loss += total_loss.item()
            epoch_recon += recon_loss.item()
            epoch_kl += kl_loss.item()

        # Средние значения за эпоху
        avg_loss = epoch_loss / len(loader)
        avg_recon = epoch_recon / len(loader)
        avg_kl = epoch_kl / len(loader)

        history['loss'].append(avg_loss)
        history['recon_loss'].append(avg_recon)
        history['kl_loss'].append(avg_kl)

        scheduler.step(avg_loss)

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1

        if avg_kl < kl_threshold:
            # Автоматически увеличиваем beta при обнаружении коллапса
            if avg_kl < critical_kl_threshold:
                kl_ratio = max(avg_kl / critical_kl_threshold, 0.001)
                beta_multiplier = 1.0 / (kl_ratio ** 0.5)
                new_beta = min(current_beta * beta_multiplier, beta * max_beta_multiplier)
            else:
                # Обычный случай: KL loss меньше порога, но не критически
                kl_ratio = max(avg_kl / kl_threshold, 0.01)  # Отношение текущего KL к порогу
                beta_multiplier = 1.0 / kl_ratio  # Множитель для beta
                new_beta = min(current_beta * beta_multiplier, beta * max_beta_multiplier)

            if avg_kl < 0.01:
                new_beta = max(new_beta, current_beta * 2.0)

            if new_beta > current_beta:
                current_beta = new_beta

        if avg_kl <= 0.0:
            current_beta = beta * max_beta_multiplier

        if verbose and ((epoch + 1) % 20 == 0 or epoch == 0 or epoch == epochs - 1):
            beta_info = f" (β={current_beta:.2f})" if current_beta > beta else ""
            print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.2f} | "
                  f"Recon: {avg_recon:.2f} | KL: {avg_kl:.2f}{beta_info}")

        if patience_counter >= early_stopping_patience:
            if verbose:
                print(f"Early stopping at epoch {epoch+1}")
            model.load_state_dict(best_model_state)
            break

    # Получение латентных представлений (используем μ, среднее)
    model.eval()
    with torch.no_grad():
        all_X = torch.tensor(X, dtype=torch.float32).to(DEVICE)
        mu, _ = model.encode(all_X)

    return mu.cpu().numpy(), model, history

## Методология обнаружения boundary кластеров

Алгоритм обнаружения промежуточных (boundary) кластеров основан на комбинации двух метрик:

### 1. Соотношение расстояний (Distance Ratio)

Для каждой точки вычисляется отношение расстояний до двух ближайших центров кластеров:

$$r = \frac{d_1}{d_2}$$

где:
- $d_1$ - расстояние до ближайшего центра кластера
- $d_2$ - расстояние до второго ближайшего центра

**Интерпретация**: чем ближе $r$ к 1, тем ближе точка находится к границе между двумя кластерами.

### 2. Энтропия соседей (Neighbor Entropy)

Анализируется разнообразие меток кластеров среди k ближайших соседей:

$$H = -\sum_{i} p_i \log p_i$$

где $p_i$ - доля соседей из кластера $i$.

**Интерпретация**: высокая энтропия означает смешанное окружение, что характерно для boundary точек.

### 3. Комбинированная метрика

Обе метрики нормализуются и комбинируются с весами:

$$score = w_d \cdot r_{norm} + w_e \cdot H_{norm}$$

По умолчанию: $w_d = 0.9$, $w_e = 0.1$.

### 4. Группировка boundary точек

Boundary точки группируются в кластеры двумя способами:
1. **По парам основных кластеров**: каждая boundary точка находится между двумя основными кластерами, поэтому логично группировать их по парам $(c_i, c_j)$
2. **DBSCAN**: если количество пар превышает лимит, используется DBSCAN для объединения похожих boundary точек


## Система метрик оценки кластеризации

Для комплексной оценки качества кластеризации используется система метрик, разделенная на две категории:

### Стандартные метрики кластеризации

Эти метрики вычисляются **только на основных кластерах** (без учета boundary кластеров):

1. **Silhouette Score** (диапазон [-1, 1], чем выше, тем лучше):
   - Измеряет, насколько хорошо точки разделены между кластерами
   - Учитывает как внутрикластерное, так и межкластерное расстояние

2. **Davies-Bouldin Index** (чем ниже, тем лучше):
   - Отношение внутрикластерного расстояния к межкластерному
   - Низкое значение означает компактные и хорошо разделенные кластеры

3. **Calinski-Harabasz Score** (чем выше, тем лучше):
   - Отношение межкластерной дисперсии к внутрикластерной
   - Высокое значение означает хорошую разделимость кластеров

### Специализированные boundary метрики

1. **boundary_ratio**: доля boundary точек от общего количества (целевой диапазон 5-15%)

2. **boundary_density**: отношение плотности основных кластеров к boundary кластерам
   - Значение близкое к 1 означает, что boundary точки достаточно плотные

3. **boundary_alignment**: среднее отношение расстояний до двух ближайших центров
   - Значение близкое к 1 означает хорошую выравненность boundary точек

4. **boundary_stability**: Обратная величина стандартного отклонения расстояний внутри boundary кластеров
   - Высокое значение означает стабильные и компактные boundary зоны

### Замечание о пространствах

**Метрики вычисляются на том пространстве, на котором работает метод**:
- VAE методы: на латентном пространстве VAE
- PCA+KMeans: на пространстве PCA
- Остальные методы: на исходном пространстве признаков


In [ ]:
# @title
def detect_boundary_clusters(latent_vectors, cluster_labels, k_neighbors=10,
                             boundary_threshold=0.5, min_boundary_points=10,
                             max_boundary_clusters=None,
                             weight_distance=0.6, weight_entropy=0.4):
    """
    Обнаруживает промежуточные (boundary) кластеры между основными кластерами.

    Алгоритм:
    1. Вычисляет интегральную метрику для каждой точки (расстояния + энтропия)
    2. Определяет boundary точки по порогу
    3. Группирует boundary точки в кластеры (по парам основных кластеров или DBSCAN)

    Параметры:
    -----------
    latent_vectors : np.ndarray
        Латентные представления данных (shape: [n_samples, latent_dim])
    cluster_labels : np.ndarray
        Метки основных кластеров (shape: [n_samples])
    k_neighbors : int
        Количество ближайших соседей для анализа энтропии
    boundary_threshold : float
        Порог для определения boundary точек (0-1). Чем выше, тем строже.
    min_boundary_points : int
        Минимальное количество точек для создания boundary кластера
    max_boundary_clusters : int, optional
        Максимальное количество boundary кластеров (None = автоматически)
    weight_distance : float
        Вес для метрики расстояний в комбинированной метрике (по умолчанию 0.6)
    weight_entropy : float
        Вес для метрики энтропии в комбинированной метрике (по умолчанию 0.4)
        Примечание: weight_distance + weight_entropy должны быть близки к 1.0

    Возвращает:
    -----------
    boundary_mask : np.ndarray
        Булева маска для boundary точек
    boundary_labels : np.ndarray
        Метки для boundary кластеров (-1 для не-boundary точек)
    """
    n_samples = len(latent_vectors)
    unique_clusters = np.unique(cluster_labels)
    n_clusters = len(unique_clusters)

    if n_clusters < 2:
        return np.zeros(n_samples, dtype=bool), np.full(n_samples, -1)

    # ========== ШАГ 1: Вычисление центров основных кластеров ==========
    cluster_centers = []
    for cluster_id in unique_clusters:
        mask = cluster_labels == cluster_id
        center = latent_vectors[mask].mean(axis=0)  # Центр = среднее точек кластера
        cluster_centers.append(center)
    cluster_centers = np.array(cluster_centers)

    # ========== ШАГ 2: Вычисление соотношения расстояний (r = d₁/d₂) ==========
    # Для каждой точки находим расстояния до всех центров кластеров
    distances_to_centers = cdist(latent_vectors, cluster_centers)

    # Находим два ближайших центра для каждой точки
    sorted_indices = np.argsort(distances_to_centers, axis=1)
    closest_center_dist = distances_to_centers[np.arange(n_samples), sorted_indices[:, 0]]      # d₁
    second_closest_center_dist = distances_to_centers[np.arange(n_samples), sorted_indices[:, 1]] # d₂

    # Вычисляем отношение расстояний: r = d₁/d₂
    # Чем ближе r к 1, тем ближе точка к границе между кластерами
    distance_ratio = closest_center_dist / (second_closest_center_dist + 1e-10)  # +1e-10 для избежания деления на 0

    # ========== ШАГ 3: Вычисление энтропии соседей (H = -Σ pᵢ log pᵢ) ==========
    # Анализ разнообразия кластеров среди k ближайших соседей
    nbrs = NearestNeighbors(n_neighbors=min(k_neighbors + 1, n_samples),
                           metric='euclidean').fit(latent_vectors)
    distances, indices = nbrs.kneighbors(latent_vectors)

    neighbor_diversity = []
    for i in range(n_samples):
        # Берем метки соседей (исключаем саму точку)
        neighbor_labels = cluster_labels[indices[i, 1:]]
        unique_neighbor_labels = np.unique(neighbor_labels)

        # Вычисляем энтропию распределения кластеров среди соседей
        label_counts = Counter(neighbor_labels)
        probs = np.array([label_counts[label] for label in unique_neighbor_labels]) / len(neighbor_labels)
        diversity = entropy(probs)  # H = -Σ pᵢ log pᵢ
        neighbor_diversity.append(diversity)

    neighbor_diversity = np.array(neighbor_diversity)

    # ========== ШАГ 4: Нормализация метрик и комбинирование ==========
    # Проверяем диапазон перед нормализацией для предотвращения деления на ноль
    # Нормализуем обе метрики в диапазон [0, 1]
    distance_range = distance_ratio.max() - distance_ratio.min()
    if distance_range > 1e-10:
        distance_ratio_norm = (distance_ratio - distance_ratio.min()) / distance_range
    else:
        # Все значения одинаковы - используем константу 0.5 (среднее значение)
        distance_ratio_norm = np.ones_like(distance_ratio) * 0.5

    diversity_range = neighbor_diversity.max() - neighbor_diversity.min()
    if diversity_range > 1e-10:
        diversity_norm = (neighbor_diversity - neighbor_diversity.min()) / diversity_range
    else:
        # Все значения одинаковы - используем константу 0.5 (среднее значение)
        diversity_norm = np.ones_like(neighbor_diversity) * 0.5

    # Нормализуем веса, чтобы их сумма была равна 1.0
    total_weight = weight_distance + weight_entropy
    if total_weight > 0:
        weight_distance_norm = weight_distance / total_weight
        weight_entropy_norm = weight_entropy / total_weight
    else:
        weight_distance_norm = 0.6
        weight_entropy_norm = 0.4

    # Комбинированная метрика: score = weight_distance·r_norm + weight_entropy·H_norm
    # Высокое значение = точка находится на границе между кластерами
    boundary_score = (distance_ratio_norm * weight_distance_norm + diversity_norm * weight_entropy_norm)

    # Определяем boundary точки по порогу
    boundary_mask = boundary_score >= boundary_threshold

    # ========== ШАГ 5: Группировка boundary точек в кластеры ==========
    boundary_labels = np.full(n_samples, -1)  # -1 означает "не boundary"

    if boundary_mask.sum() >= min_boundary_points:
        boundary_vectors = latent_vectors[boundary_mask]

        if len(boundary_vectors) > 1:
            # СТРАТЕГИЯ 1: Группировка по парам ближайших основных кластеров
            # Каждая boundary точка находится между двумя основными кластерами
            # Логично группировать их по парам (cᵢ, cⱼ)

            # Для каждой boundary точки находим два ближайших основных кластера
            distances_to_centers_boundary = cdist(boundary_vectors, cluster_centers)
            sorted_indices_boundary = np.argsort(distances_to_centers_boundary, axis=1)

            # Создаем метки на основе пар ближайших кластеров
            boundary_cluster_pairs = []
            for i in range(len(boundary_vectors)):
                closest_pair = tuple(sorted(sorted_indices_boundary[i, :2]))  # Сортируем для уникальности
                boundary_cluster_pairs.append(closest_pair)

            # Создаем уникальные метки для каждой уникальной пары кластеров
            unique_pairs = list(set(boundary_cluster_pairs))
            pair_to_label = {pair: idx + n_clusters for idx, pair in enumerate(unique_pairs)}

            # Ограничиваем максимальное количество boundary кластеров
            if max_boundary_clusters is None:
                # По умолчанию: максимум не больше количества основных кластеров
                max_boundary_clusters = n_clusters
            else:
                # Ограничиваем сверху количеством основных кластеров
                max_boundary_clusters = min(max_boundary_clusters, n_clusters)

            if len(unique_pairs) > max_boundary_clusters:
                # СТРАТЕГИЯ 2: DBSCAN для объединения похожих boundary точек
                # Если пар слишком много, используем DBSCAN

                # Автоматический подбор eps
                nbrs_boundary = NearestNeighbors(n_neighbors=min(10, len(boundary_vectors))).fit(boundary_vectors)
                distances_boundary, _ = nbrs_boundary.kneighbors(boundary_vectors)
                k_distances = np.sort(distances_boundary[:, -1])
                eps_boundary = np.percentile(k_distances, 90)  # Больший eps = меньше кластеров

                # min_samples для создания более крупных кластеров
                min_samples_boundary = max(5, len(boundary_vectors) // 20)

                # Применяем DBSCAN
                dbscan = DBSCAN(eps=eps_boundary, min_samples=min_samples_boundary)
                boundary_cluster_labels = dbscan.fit_predict(boundary_vectors)

                # Ограничиваем количество кластеров DBSCAN после его применения
                # Если DBSCAN создал больше кластеров, чем разрешено, объединяем их
                unique_dbscan_labels = np.unique(boundary_cluster_labels)
                # Исключаем шум (метка -1)
                valid_dbscan_labels = unique_dbscan_labels[unique_dbscan_labels != -1]
                n_dbscan_clusters = len(valid_dbscan_labels)

                if n_dbscan_clusters > max_boundary_clusters:
                    # Объединяем кластеры, используя иерархическую кластеризацию
                    # Создаем центры кластеров DBSCAN
                    cluster_centers = []
                    for label in valid_dbscan_labels:
                        mask = boundary_cluster_labels == label
                        if mask.sum() > 0:
                            center = boundary_vectors[mask].mean(axis=0)
                            cluster_centers.append(center)
                    cluster_centers = np.array(cluster_centers)

                    # Применяем AgglomerativeClustering для объединения кластеров
                    agg_clustering = AgglomerativeClustering(
                        n_clusters=max_boundary_clusters,
                        linkage='ward'
                    )
                    cluster_groups = agg_clustering.fit_predict(cluster_centers)

                    # Создаем маппинг: старый label -> новый label
                    label_mapping = {}
                    for old_label, new_group in zip(valid_dbscan_labels, cluster_groups):
                        label_mapping[old_label] = new_group

                    # Применяем маппинг к boundary_cluster_labels
                    boundary_cluster_labels_mapped = boundary_cluster_labels.copy()
                    for old_label, new_group in label_mapping.items():
                        boundary_cluster_labels_mapped[boundary_cluster_labels == old_label] = new_group
                    boundary_cluster_labels = boundary_cluster_labels_mapped

                # Переносим метки обратно в исходный массив
                boundary_idx = 0
                for i in range(n_samples):
                    if boundary_mask[i]:
                        if boundary_cluster_labels[boundary_idx] != -1:
                            boundary_labels[i] = boundary_cluster_labels[boundary_idx] + n_clusters
                        boundary_idx += 1
            else:
                # Используем группировку по парам кластеров
                boundary_idx = 0
                for i in range(n_samples):
                    if boundary_mask[i]:
                        pair = boundary_cluster_pairs[boundary_idx]
                        boundary_labels[i] = pair_to_label[pair]
                        boundary_idx += 1

    return boundary_mask, boundary_labels


## Интерпретация результатов кластеризации

После выполнения кластеризации важно понять характеристики каждого кластера для практического применения.

### Анализ основных кластеров

Для каждого основного кластера определяются:

1. **Важные числовые признаки**:
   - Z-score: насколько среднее значение кластера отличается от общего среднего
   - Процентное отклонение: на сколько процентов кластер отличается от среднего
   - Признаки с высоким абсолютным Z-score считаются наиболее характерными

2. **Важные категориальные признаки**:
   - Наиболее частое значение в кластере (мода)
   - Отношение частот: во сколько раз чаще значение встречается в кластере по сравнению с общим датасетом
   - Высокое отношение частот (>1.5) указывает на характерность признака для кластера

### Анализ boundary кластеров

Boundary кластеры анализируются аналогично, но их интерпретация отличается:
- Они представляют **переходные зоны** между основными кластерами
- Их характеристики часто находятся между характеристиками соседних основных кластеров
- Для бизнеса это может означать пользователей, которых нельзя однозначно отнести к какой-либо группе

### Описательная статистика

Для числовых признаков вычисляется полная описательная статистика:
- Медиана, квартили (25%, 75%), среднее, стандартное отклонение
- Это позволяет понять распределение значений внутри каждого кластера


## Система метрик

Вычисляет как стандартные метрики кластеризации, так и специализированные boundary-метрики.


In [ ]:
# @title
def evaluate_clustering_metrics(data_space, cluster_labels, boundary_labels):
    """
    Вычисляет комплексную систему метрик для кластеризации с boundary кластерами.

    Метрики вычисляются на переданном пространстве данных (data_space).
    Это может быть исходное пространство признаков, латентное пространство VAE,
    пространство PCA и т.д. При сравнении методов каждый метод оценивается
    на своем соответствующем пространстве (метод + его пространство).

    Основные метрики (стандартные):
    - Silhouette Score: мера разделимости кластеров (чем выше, тем лучше, диапазон [-1, 1])
    - Davies-Bouldin Index: отношение внутрикластерного расстояния к межкластерному (чем ниже, тем лучше)
    - Calinski-Harabasz Score: отношение межкластерной дисперсии к внутрикластерной (чем выше, тем лучше)

    Boundary метрики (специализированные):
    - boundary_ratio: доля boundary-точек
    - boundary_density: отношение плотности основных кластеров к boundary-кластерам
    - boundary_alignment: среднее отношение расстояний до двух ближайших центров (выравненность)
    - boundary_stability: обратная величина стандартного отклонения расстояний внутри boundary-кластеров

    Метрики вычисляются только на основных кластерах (без учета boundary кластеров).

    Параметры:
    -----------
    data_space : np.ndarray
        Пространство данных для вычисления метрик (может быть исходное, латентное, PCA и т.д.)
    cluster_labels : np.ndarray
        Метки основных кластеров
    boundary_labels : np.ndarray
        Метки boundary кластеров
    """
    metrics = {}

    # Разделяем основные и boundary точки
    main_mask = boundary_labels == -1
    boundary_mask = boundary_labels != -1

    main_labels = cluster_labels[main_mask].copy() if main_mask.sum() > 0 else np.array([])
    main_vectors = data_space[main_mask] if main_mask.sum() > 0 else np.array([]).reshape(0, data_space.shape[1])

    # ========== ОСНОВНЫЕ МЕТРИКИ  ==========
    if len(np.unique(cluster_labels)) > 1 and len(main_vectors) > 0:
        if len(np.unique(main_labels)) > 1 and len(main_vectors) > 1:
            metrics['silhouette'] = silhouette_score(main_vectors, main_labels)
            metrics['davies_bouldin'] = davies_bouldin_score(main_vectors, main_labels)
            metrics['calinski_harabasz'] = calinski_harabasz_score(main_vectors, main_labels)
        else:
            metrics['silhouette'] = -1.0
            metrics['davies_bouldin'] = float('inf')
            metrics['calinski_harabasz'] = 0.0
    else:
        metrics['silhouette'] = -1.0
        metrics['davies_bouldin'] = float('inf')
        metrics['calinski_harabasz'] = 0.0

    # ========== BOUNDARY МЕТРИКИ ==========
    n_boundary = boundary_mask.sum()
    n_total = len(cluster_labels)
    metrics['boundary_ratio'] = n_boundary / n_total if n_total > 0 else 0.0

    if n_boundary > 0:
        boundary_vectors = data_space[boundary_mask]

        # 1. Boundary Density: отношение плотности основных кластеров к boundary
        if len(boundary_vectors) > 1:
            boundary_distances = cdist(boundary_vectors, boundary_vectors)
            np.fill_diagonal(boundary_distances, np.inf)
            avg_boundary_distance = np.min(boundary_distances, axis=1).mean()

            if len(main_vectors) > 1:
                main_distances = cdist(main_vectors, main_vectors)
                np.fill_diagonal(main_distances, np.inf)
                avg_main_distance = np.min(main_distances, axis=1).mean()
                # Чем ближе к 1, тем лучше (boundary точки должны быть плотными)
                metrics['boundary_density'] = avg_main_distance / (avg_boundary_distance + 1e-10)
            else:
                metrics['boundary_density'] = 0.0
        else:
            metrics['boundary_density'] = 0.0

        # 2. Boundary Alignment: выравненность относительно ближайших центров
        unique_main_clusters = np.unique(cluster_labels[main_mask])
        cluster_centers = []
        for cluster_id in unique_main_clusters:
            mask = (cluster_labels == cluster_id) & main_mask
            center = data_space[mask].mean(axis=0)
            cluster_centers.append(center)
        cluster_centers = np.array(cluster_centers)

        distances_to_centers = cdist(boundary_vectors, cluster_centers)
        sorted_indices = np.argsort(distances_to_centers, axis=1)
        closest_dist = distances_to_centers[np.arange(len(boundary_vectors)), sorted_indices[:, 0]]
        second_closest_dist = distances_to_centers[np.arange(len(boundary_vectors)), sorted_indices[:, 1]]
        # Чем ближе отношение к 1, тем лучше выравнена точка
        alignment_ratios = closest_dist / (second_closest_dist + 1e-10)
        metrics['boundary_alignment'] = alignment_ratios.mean()

        # 3. Boundary Stability: стабильность boundary зон
        unique_boundary_clusters = np.unique(boundary_labels[boundary_mask])
        stability_scores = []
        for boundary_cluster_id in unique_boundary_clusters:
            mask = boundary_labels == boundary_cluster_id
            if mask.sum() > 1:
                cluster_points = data_space[mask]
                center = cluster_points.mean(axis=0)
                distances = np.linalg.norm(cluster_points - center, axis=1)
                # Стабильность = обратная величина стандартного отклонения
                stability = 1.0 / (distances.std() + 1e-10)
                stability_scores.append(stability)
        metrics['boundary_stability'] = np.mean(stability_scores) if stability_scores else 0.0
    else:
        metrics['boundary_density'] = 0.0
        metrics['boundary_alignment'] = 0.0
        metrics['boundary_stability'] = 0.0

    return metrics


## Методология сравнения методов

Для объективной оценки нашего метода (VAE + KMeans + Boundary) мы сравниваем его с классическими методами кластеризации:

### Baseline методы

1. **PCA + KMeans**: линейное снижение размерности с последующей кластеризацией
2. **Standard KMeans**: кластеризация на исходном пространстве признаков
3. **DBSCAN**: плотностная кластеризация, автоматически определяющая количество кластеров
4. **Agglomerative Clustering**: иерархическая кластеризация с фиксированным количеством кластеров

### Важные методологические принципы

1. **Справедливое сравнение**:
   - Все методы используют одинаковое количество кластеров (определяется из результатов VAE)
   - Baseline методы **не используют** boundary detection, так как это часть предлагаемого метода

2. **Пространства для метрик**:
   - Каждый метод оценивается на своем соответствующем пространстве
   - Это сравнение "метод + его пространство", что отражает реальную производительность подхода

3. **Визуализация**:
   - Все методы визуализируются через PCA в 2D/3D для единообразия
   - Это позволяет визуально сравнить качество разделения кластеров


In [ ]:
# @title
def analyze_cluster_characteristics(df_original, cluster_labels, boundary_labels,
                                   preprocessor, feature_info, top_n=5):
    """
    Анализирует характеристики кластеров и определяет наиболее важные признаки.

    Параметры:
    -----------
    df_original : pd.DataFrame
        Исходный DataFrame (до предобработки)
    cluster_labels : np.ndarray
        Метки основных кластеров
    boundary_labels : np.ndarray
        Метки boundary кластеров
    preprocessor : ColumnTransformer
        Обученный препроцессор
    feature_info : dict
        Информация о признаках
    top_n : int
        Количество топ признаков для вывода

    Возвращает:
    -----------
    cluster_descriptions : dict
        Описание каждого кластера с важными признаками
    """
    # Получаем числовые и категориальные признаки
    numeric_features = feature_info.get('numeric_features', [])
    categorical_features = feature_info.get('categorical_features', [])

    # Создаем копию исходных данных
    df = df_original.copy()

    # Исключаем колонки, которые не были использованы
    all_features = numeric_features + categorical_features
    df_analysis = df[all_features].copy() if all_features else df.copy()

    # Добавляем метки кластеров
    df_analysis['_cluster'] = cluster_labels
    df_analysis['_boundary'] = boundary_labels
    df_analysis['_is_boundary'] = (boundary_labels != -1)

    cluster_descriptions = {}

    # Анализ основных кластеров
    unique_main_clusters = np.unique(cluster_labels[boundary_labels == -1])

    for cluster_id in unique_main_clusters:
        mask = (cluster_labels == cluster_id) & (boundary_labels == -1)
        cluster_data = df_analysis[mask]

        if len(cluster_data) == 0:
            continue

        description = {
            'cluster_id': cluster_id,
            'type': 'main',
            'size': len(cluster_data),
            'percentage': len(cluster_data) / len(df_analysis) * 100,
            'important_features': {}
        }

        # Анализ числовых признаков
        for feature in numeric_features:
            if feature not in cluster_data.columns:
                continue

            cluster_mean = cluster_data[feature].mean()
            overall_mean = df_analysis[feature].mean()
            overall_std = df_analysis[feature].std()

            if overall_std > 0:
                # Z-score: насколько среднее кластера отличается от общего среднего
                z_score = (cluster_mean - overall_mean) / overall_std
                # Процентное отклонение
                pct_diff = ((cluster_mean - overall_mean) / abs(overall_mean) * 100) if overall_mean != 0 else 0

                description['important_features'][feature] = {
                    'cluster_mean': cluster_mean,
                    'overall_mean': overall_mean,
                    'z_score': z_score,
                    'pct_diff': pct_diff,
                    'type': 'numeric'
                }

        # Анализ категориальных признаков
        for feature in categorical_features:
            if feature not in cluster_data.columns:
                continue

            # Наиболее частое значение в кластере
            cluster_mode = cluster_data[feature].mode()
            if len(cluster_mode) > 0:
                mode_value = cluster_mode[0]
                mode_freq = (cluster_data[feature] == mode_value).sum() / len(cluster_data)

                # Частота этого значения в общем датасете
                overall_freq = (df_analysis[feature] == mode_value).sum() / len(df_analysis)

                # Отношение частот (если > 1, то признак более характерен для кластера)
                freq_ratio = mode_freq / (overall_freq + 1e-10)

                description['important_features'][feature] = {
                    'mode_value': mode_value,
                    'mode_freq_in_cluster': mode_freq * 100,
                    'overall_freq': overall_freq * 100,
                    'freq_ratio': freq_ratio,
                    'type': 'categorical'
                }

        # Сортируем признаки по важности (по абсолютному z-score для числовых, по freq_ratio для категориальных)
        sorted_features = sorted(
            description['important_features'].items(),
            key=lambda x: abs(x[1].get('z_score', 0)) if x[1]['type'] == 'numeric' else x[1].get('freq_ratio', 0),
            reverse=True
        )

        description['top_features'] = [f[0] for f in sorted_features[:top_n]]
        cluster_descriptions[cluster_id] = description

    # Анализ boundary кластеров
    unique_boundary_clusters = np.unique(boundary_labels[boundary_labels != -1])

    for boundary_id in unique_boundary_clusters:
        mask = boundary_labels == boundary_id
        cluster_data = df_analysis[mask]

        if len(cluster_data) == 0:
            continue

        description = {
            'cluster_id': boundary_id,
            'type': 'boundary',
            'size': len(cluster_data),
            'percentage': len(cluster_data) / len(df_analysis) * 100,
            'important_features': {}
        }

        # Анализ числовых признаков
        for feature in numeric_features:
            if feature not in cluster_data.columns:
                continue

            cluster_mean = cluster_data[feature].mean()
            overall_mean = df_analysis[feature].mean()
            overall_std = df_analysis[feature].std()

            if overall_std > 0:
                z_score = (cluster_mean - overall_mean) / overall_std
                pct_diff = ((cluster_mean - overall_mean) / abs(overall_mean) * 100) if overall_mean != 0 else 0

                description['important_features'][feature] = {
                    'cluster_mean': cluster_mean,
                    'overall_mean': overall_mean,
                    'z_score': z_score,
                    'pct_diff': pct_diff,
                    'type': 'numeric'
                }

        # Анализ категориальных признаков
        for feature in categorical_features:
            if feature not in cluster_data.columns:
                continue

            cluster_mode = cluster_data[feature].mode()
            if len(cluster_mode) > 0:
                mode_value = cluster_mode[0]
                mode_freq = (cluster_data[feature] == mode_value).sum() / len(cluster_data)
                overall_freq = (df_analysis[feature] == mode_value).sum() / len(df_analysis)
                freq_ratio = mode_freq / (overall_freq + 1e-10)

                description['important_features'][feature] = {
                    'mode_value': mode_value,
                    'mode_freq_in_cluster': mode_freq * 100,
                    'overall_freq': overall_freq * 100,
                    'freq_ratio': freq_ratio,
                    'type': 'categorical'
                }

        sorted_features = sorted(
            description['important_features'].items(),
            key=lambda x: abs(x[1].get('z_score', 0)) if x[1]['type'] == 'numeric' else x[1].get('freq_ratio', 0),
            reverse=True
        )

        description['top_features'] = [f[0] for f in sorted_features[:top_n]]
        cluster_descriptions[f'boundary_{boundary_id}'] = description

    return cluster_descriptions


In [ ]:
# @title
def get_cluster_statistics(df_original, cluster_labels, boundary_labels, feature_info):
    """
    Вычисляет описательную статистику по числовым признакам для каждого кластера.

    Параметры:
    -----------
    df_original : pd.DataFrame
        Исходный DataFrame (до предобработки)
    cluster_labels : np.ndarray
        Метки основных кластеров
    boundary_labels : np.ndarray
        Метки boundary кластеров
    feature_info : dict
        Информация о признаках

    Возвращает:
    -----------
    statistics : dict
        Словарь со статистикой для каждого кластера
    """
    numeric_features = feature_info.get('numeric_features', [])

    if not numeric_features:
        return {}

    # Создаем копию исходных данных только с числовыми признаками
    df = df_original[numeric_features].copy()

    # Добавляем метки кластеров
    df['_cluster'] = cluster_labels
    df['_boundary'] = boundary_labels
    df['_is_boundary'] = (boundary_labels != -1)

    statistics = {}

    # Статистика для основных кластеров
    unique_main_clusters = np.unique(cluster_labels[boundary_labels == -1])

    for cluster_id in unique_main_clusters:
        mask = (cluster_labels == cluster_id) & (boundary_labels == -1)
        cluster_data = df[mask][numeric_features]

        if len(cluster_data) == 0:
            continue

        cluster_stats = {}
        for feature in numeric_features:
            if feature not in cluster_data.columns:
                continue

            values = cluster_data[feature].dropna()
            if len(values) > 0:
                cluster_stats[feature] = {
                    'count': len(values),
                    'mean': values.mean(),
                    'std': values.std(),
                    'min': values.min(),
                    '25%': values.quantile(0.25),
                    'median': values.median(),
                    '75%': values.quantile(0.75),
                    'max': values.max()
                }

        statistics[f'Cluster_{cluster_id}'] = {
            'type': 'main',
            'size': len(cluster_data),
            'features': cluster_stats
        }

    # Статистика для boundary кластеров
    unique_boundary_clusters = np.unique(boundary_labels[boundary_labels != -1])

    for boundary_id in unique_boundary_clusters:
        mask = boundary_labels == boundary_id
        cluster_data = df[mask][numeric_features]

        if len(cluster_data) == 0:
            continue

        cluster_stats = {}
        for feature in numeric_features:
            if feature not in cluster_data.columns:
                continue

            values = cluster_data[feature].dropna()
            if len(values) > 0:
                cluster_stats[feature] = {
                    'count': len(values),
                    'mean': values.mean(),
                    'std': values.std(),
                    'min': values.min(),
                    '25%': values.quantile(0.25),
                    'median': values.median(),
                    '75%': values.quantile(0.75),
                    'max': values.max()
                }

        statistics[f'Boundary_{boundary_id}'] = {
            'type': 'boundary',
            'size': len(cluster_data),
            'features': cluster_stats
        }

    return statistics


In [ ]:
# @title
def print_cluster_descriptions(cluster_descriptions, statistics, verbose=True):
    """
    Выводит описание кластеров и статистику в читаемом формате.

    Параметры:
    -----------
    cluster_descriptions : dict
        Описание кластеров из analyze_cluster_characteristics
    statistics : dict
        Статистика из get_cluster_statistics
    verbose : bool
        Выводить ли подробную информацию
    """
    if not verbose:
        return

    # Сортируем кластеры: сначала основные, потом boundary
    main_clusters = {k: v for k, v in cluster_descriptions.items() if v['type'] == 'main'}
    boundary_clusters = {k: v for k, v in cluster_descriptions.items() if v['type'] == 'boundary'}

    print("\n" + "─"*80)
    print("ОСНОВНЫЕ КЛАСТЕРЫ")
    print("─"*80)

    for cluster_id in sorted(main_clusters.keys()):
        desc = main_clusters[cluster_id]
        print(f"\nКластер {cluster_id}")
        print(f"   Размер: {desc['size']} объектов ({desc['percentage']:.2f}% от общего количества)")
        print(f"   Топ-{len(desc['top_features'])} важных признаков:")

        for i, feature in enumerate(desc['top_features'], 1):
            feat_info = desc['important_features'][feature]
            if feat_info['type'] == 'numeric':
                z_score = feat_info['z_score']
                pct_diff = feat_info['pct_diff']
                direction = "выше" if z_score > 0 else "ниже"
                print(f"      {i}. {feature}:")
                print(f"         - Среднее значение: {feat_info['cluster_mean']:.4f} "
                      f"(общее среднее: {feat_info['overall_mean']:.4f})")
                print(f"         - Отклонение: {abs(pct_diff):.2f}% {direction} среднего")
                print(f"         - Z-score: {z_score:.2f}")
            else:
                mode_value = feat_info['mode_value']
                freq_ratio = feat_info['freq_ratio']
                print(f"      {i}. {feature}:")
                print(f"         - Наиболее частое значение: '{mode_value}' "
                      f"({feat_info['mode_freq_in_cluster']:.1f}% в кластере vs "
                      f"{feat_info['overall_freq']:.1f}% в целом)")
                print(f"         - Отношение частот: {freq_ratio:.2f}x")

    # Выводим boundary кластеры
    if boundary_clusters:
        print("\n" + "─"*80)
        print("BOUNDARY КЛАСТЕРЫ")
        print("─"*80)

        for cluster_key in sorted(boundary_clusters.keys()):
            desc = boundary_clusters[cluster_key]
            boundary_id = desc['cluster_id']
            print(f"\nBoundary кластер {boundary_id}")
            print(f"   Размер: {desc['size']} объектов ({desc['percentage']:.2f}% от общего количества)")
            print(f"   Топ-{len(desc['top_features'])} важных признаков:")

            for i, feature in enumerate(desc['top_features'], 1):
                feat_info = desc['important_features'][feature]
                if feat_info['type'] == 'numeric':
                    z_score = feat_info['z_score']
                    pct_diff = feat_info['pct_diff']
                    direction = "выше" if z_score > 0 else "ниже"
                    print(f"      {i}. {feature}:")
                    print(f"         - Среднее значение: {feat_info['cluster_mean']:.4f} "
                          f"(общее среднее: {feat_info['overall_mean']:.4f})")
                    print(f"         - Отклонение: {abs(pct_diff):.2f}% {direction} среднего")
                    print(f"         - Z-score: {z_score:.2f}")
                else:
                    mode_value = feat_info['mode_value']
                    freq_ratio = feat_info['freq_ratio']
                    print(f"      {i}. {feature}:")
                    print(f"         - Наиболее частое значение: '{mode_value}' "
                          f"({feat_info['mode_freq_in_cluster']:.1f}% в кластере vs "
                          f"{feat_info['overall_freq']:.1f}% в целом)")
                    print(f"         - Отношение частот: {freq_ratio:.2f}x")

    # Выводим описательную статистику
    if statistics:
        print("\n" + "="*80)
        print("ОПИСАТЕЛЬНАЯ СТАТИСТИКА ПО ЧИСЛОВЫМ ПРИЗНАКАМ")
        print("="*80)

        for cluster_name, cluster_stats in statistics.items():
            if not cluster_stats['features']:
                continue

            cluster_type = "Основной" if cluster_stats['type'] == 'main' else "Boundary"
            print(f"\n{cluster_type} кластер: {cluster_name} (размер: {cluster_stats['size']})")
            print("─"*80)

            # Создаем DataFrame для красивого вывода
            stats_data = []
            for feature, stats in cluster_stats['features'].items():
                stats_data.append({
                    'Признак': feature,
                    'Среднее': f"{stats['mean']:.4f}",
                    'Стд. откл.': f"{stats['std']:.4f}",
                    'Мин': f"{stats['min']:.4f}",
                    '25%': f"{stats['25%']:.4f}",
                    'Медиана': f"{stats['median']:.4f}",
                    '75%': f"{stats['75%']:.4f}",
                    'Макс': f"{stats['max']:.4f}"
                })

            stats_df = pd.DataFrame(stats_data)
            print(stats_df.to_string(index=False))

    print("\n" + "="*80)


In [ ]:
# @title
def create_baseline_methods(X, results, random_state=None, compute_metrics=False, measure_time=False):
    """
    Создает baseline методы для сравнения: PCA+KMeans, Standard KMeans, DBSCAN, Agglomerative, VAE+KMeans.

    Эта функция устраняет дубликаты кода, которые повторялись в нескольких местах.

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные
    results : dict
        Результаты VAE + KMeans + Boundary метода
    random_state : int, optional
        Random state для воспроизводимости (по умолчанию использует RANDOM_STATE)
    compute_metrics : bool
        Вычислять ли метрики для каждого метода (для visualize_metrics_comparison)
    measure_time : bool
        Измерять ли время выполнения каждого метода

    Возвращает:
    -----------
    methods_data : list
        Список кортежей (method_name, vectors, labels) для визуализации
    methods_metrics : list, optional
        Список словарей с метриками для каждого метода (если compute_metrics=True)
    """
    if random_state is None:
        random_state = RANDOM_STATE

    vae_latent = results['latent_vectors']
    vae_clusters = results['cluster_labels']
    n_clusters_vae = len(np.unique(vae_clusters))

    methods_data = []
    methods_metrics = [] if compute_metrics else None

    # 1. PCA + KMeans
    start_time = time.time() if measure_time else None
    pca = PCA(n_components=min(15, X.shape[1]), random_state=random_state)
    X_pca = pca.fit_transform(X)
    kmeans_pca = KMeans(n_clusters=n_clusters_vae, n_init=10, random_state=random_state)
    labels_pca = kmeans_pca.fit_predict(X_pca)
    pca_time = time.time() - start_time if measure_time else None
    methods_data.append(('PCA + KMeans', X_pca, labels_pca))

    if compute_metrics:
        boundary_labels_pca = np.full(len(labels_pca), -1)
        # Метрики вычисляются на пространстве PCA (X_pca)
        metrics_pca = evaluate_clustering_metrics(X_pca, labels_pca, boundary_labels_pca)
        metrics_pca['method'] = 'PCA + KMeans'
        if measure_time:
            metrics_pca['execution_time'] = pca_time
        methods_metrics.append(metrics_pca)

    # 2. Standard KMeans
    start_time = time.time() if measure_time else None
    kmeans_std = KMeans(n_clusters=n_clusters_vae, n_init=10, random_state=random_state)
    labels_std = kmeans_std.fit_predict(X)
    kmeans_time = time.time() - start_time if measure_time else None
    methods_data.append(('Standard KMeans', X, labels_std))

    if compute_metrics:
        boundary_labels_std = np.full(len(labels_std), -1)
        # Метрики вычисляются на исходном пространстве (X)
        metrics_std = evaluate_clustering_metrics(X, labels_std, boundary_labels_std)
        metrics_std['method'] = 'Standard KMeans'
        if measure_time:
            metrics_std['execution_time'] = kmeans_time
        methods_metrics.append(metrics_std)

    # 3. DBSCAN
    start_time = time.time() if measure_time else None
    nbrs = NearestNeighbors(n_neighbors=4).fit(X)
    distances, _ = nbrs.kneighbors(X)
    k_distances = np.sort(distances[:, -1])
    eps = np.percentile(k_distances, 95)
    dbscan = DBSCAN(eps=eps, min_samples=30)
    labels_dbscan = dbscan.fit_predict(X)
    dbscan_time = time.time() - start_time if measure_time else None
    methods_data.append(('DBSCAN', X, labels_dbscan))

    if compute_metrics:
        boundary_labels_dbscan = np.full(len(labels_dbscan), -1)
        # Метрики вычисляются на исходном пространстве (X)
        metrics_dbscan = evaluate_clustering_metrics(X, labels_dbscan, boundary_labels_dbscan)
        metrics_dbscan['method'] = 'DBSCAN'
        if measure_time:
            metrics_dbscan['execution_time'] = dbscan_time
        methods_metrics.append(metrics_dbscan)

    # 4. Agglomerative
    start_time = time.time() if measure_time else None
    agg = AgglomerativeClustering(n_clusters=n_clusters_vae, linkage='ward')
    labels_agg = agg.fit_predict(X)
    agg_time = time.time() - start_time if measure_time else None
    methods_data.append(('Agglomerative', X, labels_agg))

    if compute_metrics:
        boundary_labels_agg = np.full(len(labels_agg), -1)
        # Метрики вычисляются на исходном пространстве (X)
        metrics_agg = evaluate_clustering_metrics(X, labels_agg, boundary_labels_agg)
        metrics_agg['method'] = 'Agglomerative'
        if measure_time:
            metrics_agg['execution_time'] = agg_time
        methods_metrics.append(metrics_agg)

    # 5. VAE + KMeans (время уже измерено в run_vae_boundary_clustering)
    methods_data.append(('VAE + KMeans', vae_latent, vae_clusters))

    if compute_metrics:
        # Используем правильные boundary_labels из results
        vae_boundary_labels = results.get('boundary_labels', np.full(len(vae_clusters), -1))
        # Метрики вычисляются на латентном пространстве VAE (vae_latent)
        metrics_vae = evaluate_clustering_metrics(vae_latent, vae_clusters, vae_boundary_labels)
        metrics_vae['method'] = 'VAE + KMeans + Boundary'
        if measure_time:
            if 'execution_time' in results and results['execution_time'] is not None:
                metrics_vae['execution_time'] = results['execution_time']
            else:
                metrics_vae['execution_time'] = np.nan
        methods_metrics.append(metrics_vae)

    if compute_metrics:
        return methods_data, methods_metrics
    else:
        return methods_data

print("Вспомогательная функция для создания методов сравнения определена")


Вспомогательная функция для создания методов сравнения определена


## Визуализация результатов

Визуализация играет важную роль в понимании и интерпретации результатов кластеризации.

### Типы визуализаций

1. **2D/3D проекции**:
   - Используется PCA для снижения размерности до 2D/3D
   - Позволяет визуально оценить качество разделения кластеров
   - Основные кластеры и boundary кластеры визуализируются разными цветами и маркерами

2. **Сравнение метрик**:
   - Столбчатые диаграммы для сравнения методов
   - Показывает стандартные метрики (silhouette, Davies-Bouldin, Calinski-Harabasz)

3. **Тепловые карты p-values**:
   - Визуализация статистической значимости различий между методами
   - Зеленый цвет = значимые различия (p < 0.05)
   - Серый цвет = незначимые различия


## Workflow алгоритма кластеризации

Главная функция `run_vae_boundary_clustering` выполняет полный pipeline кластеризации:

### Этап 1: Предобработка данных
- Автоматическое определение типов признаков
- Нормализация и кодирование
- Проверка корректности данных

### Этап 2: Оптимизация гиперпараметров (опционально)
- Bayesian Optimization для поиска оптимальных параметров
- Используется train/validation split для предотвращения переобучения
- Оптимизируются: `latent_dim`, `n_clusters`, `boundary_threshold`, `vae_beta`
- Целевая функция: максимизация silhouette score на validation set

### Этап 3: Обучение VAE
- Обучение модели на всех данных (после оптимизации)
- Получение латентных представлений для кластеризации
- Мониторинг процесса обучения (loss, early stopping)

### Этап 4: Кластеризация KMeans
- Применение KMeans на латентном пространстве VAE
- Определение основных кластеров

### Этап 5: Обнаружение boundary кластеров
- Вычисление интегральной метрики для каждой точки
- Определение boundary точек по порогу
- Группировка boundary точек в кластеры

### Этап 6: Вычисление метрик и анализ
- Стандартные метрики кластеризации
- Специализированные boundary метрики
- Анализ характеристик кластеров


## Визуализации


In [ ]:
# @title
def visualize_methods_comparison_2d_no_boundary(X, results):
    """
    Визуализирует 2D кластеризацию всех методов БЕЗ boundary кластеров.

    Показывает только основные кластеры для:
    - PCA + KMeans
    - Standard KMeans
    - DBSCAN
    - Agglomerative
    - VAE + KMeans (наш метод, без boundary)

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные
    results : dict
        Результаты VAE + KMeans + Boundary метода
    """
    methods_data = create_baseline_methods(X, results)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    axes = axes.flatten()

    for idx, (method_name, vectors, labels) in enumerate(methods_data):
        if idx >= 6:
            break

        ax = axes[idx]

        # PCA для визуализации в 2D
        pca_2d = PCA(n_components=2, random_state=RANDOM_STATE)
        vectors_2d = pca_2d.fit_transform(vectors)

        # Определяем цвета для основных кластеров
        unique_clusters = np.unique(labels)
        # Исключаем шум (метка -1 для DBSCAN)
        unique_clusters = unique_clusters[unique_clusters != -1]

        main_colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))

        # Рисуем основные кластеры
        for i, cid in enumerate(unique_clusters):
            mask = labels == cid
            if mask.sum() > 0:
                ax.scatter(vectors_2d[mask, 0], vectors_2d[mask, 1],
                          c=[main_colors[i]], label=f'Cluster {cid}',
                          alpha=0.6, s=15, edgecolors='black', linewidths=0.3)

        # Для DBSCAN показываем шум отдельно
        if method_name == 'DBSCAN' and -1 in labels:
            noise_mask = labels == -1
            if noise_mask.sum() > 0:
                ax.scatter(vectors_2d[noise_mask, 0], vectors_2d[noise_mask, 1],
                          c='gray', label='Noise', alpha=0.3, s=10, marker='x')

        ax.set_xlabel('PC1', fontsize=10)
        ax.set_ylabel('PC2', fontsize=10)
        ax.set_title(method_name, fontsize=12, fontweight='bold')
        ax.legend(loc='upper right', fontsize=8)
        ax.grid(True, alpha=0.3)

    # Скрываем последний subplot
    axes[5].axis('off')

    plt.suptitle('Сравнение методов кластеризации (только основные кластеры, без boundary)',
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

    return fig


In [ ]:
# @title
def visualize_metrics_comparison(X, results, comparison_df=None):
    """
    Визуализирует сравнение метрик базовых методов (без boundary) и нашего метода.

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные
    results : dict
        Результаты VAE + KMeans + Boundary метода
    comparison_df : pd.DataFrame, optional
        DataFrame с метриками (если уже вычислен). Если None, вычисляется автоматически.
    """
    if comparison_df is None:
        _, methods_metrics = create_baseline_methods(X, results, compute_metrics=True, measure_time=True)
        comparison_df = pd.DataFrame(methods_metrics)

    # Основные метрики для сравнения
    metrics_to_plot = ['silhouette', 'davies_bouldin', 'calinski_harabasz']

    # Создаем фигуру с subplots (3 метрики + время выполнения)
    fig, axes = plt.subplots(1, 4, figsize=(24, 5))

    methods = comparison_df['method'].values

    # 1. Silhouette Score
    ax = axes[0]
    values = comparison_df['silhouette'].values
    colors = ['green' if 'VAE' in m else 'steelblue' for m in methods]
    bars = ax.barh(methods, values, color=colors, alpha=0.7)
    ax.set_xlabel('Silhouette Score', fontsize=12)
    ax.set_title('Silhouette Score\n(чем выше, тем лучше)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    # Добавляем значения на столбцы
    for i, (bar, val) in enumerate(zip(bars, values)):
        ax.text(val + 0.01, i, f'{val:.3f}', va='center', fontsize=9)

    # 2. Davies-Bouldin Index
    ax = axes[1]
    values = comparison_df['davies_bouldin'].values
    colors = ['green' if 'VAE' in m else 'steelblue' for m in methods]
    bars = ax.barh(methods, values, color=colors, alpha=0.7)
    ax.set_xlabel('Davies-Bouldin Index', fontsize=12)
    ax.set_title('Davies-Bouldin Index\n(чем ниже, тем лучше)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    # Добавляем значения на столбцы
    for i, (bar, val) in enumerate(zip(bars, values)):
        ax.text(val + 0.05, i, f'{val:.3f}', va='center', fontsize=9)

    # 3. Calinski-Harabasz Score
    ax = axes[2]
    values = comparison_df['calinski_harabasz'].values
    colors = ['green' if 'VAE' in m else 'steelblue' for m in methods]
    bars = ax.barh(methods, values, color=colors, alpha=0.7)
    ax.set_xlabel('Calinski-Harabasz Score', fontsize=12)
    ax.set_title('Calinski-Harabasz Score\n(чем выше, тем лучше)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    # Добавляем значения на столбцы (форматируем большие числа)
    for i, (bar, val) in enumerate(zip(bars, values)):
        if val > 1000:
            label = f'{val/1000:.1f}K'
        else:
            label = f'{val:.0f}'
        ax.text(val + val*0.02, i, label, va='center', fontsize=9)

    # 4. Время выполнения
    ax = axes[3]
    if 'execution_time' in comparison_df.columns:
        values = comparison_df['execution_time'].values
        values_plot = np.where(np.isnan(values), 0, values)
        colors = ['green' if 'VAE' in m else 'steelblue' for m in methods]
        bars = ax.barh(methods, values_plot, color=colors, alpha=0.7)
        ax.set_xlabel('Время выполнения (сек)', fontsize=12)
        ax.set_title('Время выполнения\n(чем меньше, тем лучше)', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        # Добавляем значения на столбцы
        for i, (bar, val) in enumerate(zip(bars, values)):
            if pd.isna(val) or np.isnan(val):
                label = 'N/A'
                ax.text(0.01, i, label, va='center', fontsize=9, color='red', style='italic')
            else:
                if val > 60:
                    label = f'{val/60:.1f} мин'
                else:
                    label = f'{val:.2f} сек'
                ax.text(val + val*0.02, i, label, va='center', fontsize=9)
    else:
        ax.axis('off')
        ax.text(0.5, 0.5, 'Время выполнения\nне измерено',
                ha='center', va='center', fontsize=12, transform=ax.transAxes)

    plt.suptitle('Сравнение метрик кластеризации и времени выполнения',
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    return fig, comparison_df


In [ ]:
# @title
def visualize_methods_comparison_3d(X, results):
    """
    Визуализирует 3D кластеризацию всех методов.

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные
    results : dict
        Результаты VAE + KMeans + Boundary метода
    """
    methods_data = create_baseline_methods(X, results)

    fig = plt.figure(figsize=(24, 12))

    for idx, (method_name, vectors, labels) in enumerate(methods_data):
        ax = fig.add_subplot(2, 3, idx+1, projection='3d')

        # PCA для визуализации в 3D
        pca_3d = PCA(n_components=3, random_state=RANDOM_STATE)
        vectors_3d = pca_3d.fit_transform(vectors)

        unique_clusters = np.unique(labels)
        unique_clusters = unique_clusters[unique_clusters != -1]
        main_colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))

        for i, cid in enumerate(unique_clusters):
            mask = labels == cid
            if mask.sum() > 0:
                ax.scatter(vectors_3d[mask, 0], vectors_3d[mask, 1], vectors_3d[mask, 2],
                          c=[main_colors[i]], label=f'Cluster {cid}',
                          alpha=0.6, s=15, edgecolors='black', linewidths=0.3)

        if method_name == 'DBSCAN' and -1 in labels:
            noise_mask = labels == -1
            if noise_mask.sum() > 0:
                ax.scatter(vectors_3d[noise_mask, 0], vectors_3d[noise_mask, 1], vectors_3d[noise_mask, 2],
                          c='gray', label='Noise', alpha=0.3, s=10, marker='x')

        ax.set_xlabel('PC1', fontsize=10)
        ax.set_ylabel('PC2', fontsize=10)
        ax.set_zlabel('PC3', fontsize=10)
        ax.set_title(method_name, fontsize=12, fontweight='bold')
        ax.legend(loc='upper right', fontsize=8)

    plt.suptitle('Сравнение методов кластеризации в 3D', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

    return fig

print("Функция 3D визуализации методов сравнения определена")


Функция 3D визуализации методов сравнения определена


## Главная функция для запуска всего кода

Универсальная функция, которая принимает DataFrame и автоматически выполняет весь процесс кластеризации.


In [ ]:
# @title
def run_vae_boundary_clustering(df, exclude_columns=None,
                                latent_dim=15, hidden_dims=[512, 256, 128],
                                n_clusters=6, boundary_threshold=0.5,
                                max_boundary_clusters=None,
                                vae_epochs=150, vae_beta=4.0,
                                vae_batch_size=256, vae_lr=1e-4, vae_dropout=0.2,
                                use_batch_norm=True, early_stopping_patience=20,
                                optimize=False, max_optimization_iterations=10,
                                n_clusters_range=None, verbose=True):
    """
    Главная функция для запуска VAE + KMeans + Boundary кластеризации.

    Параметры:
    -----------
    df : pd.DataFrame
        Входной DataFrame с данными
    exclude_columns : list, optional
        Колонки для исключения из обработки
    latent_dim : int
        Размерность латентного пространства VAE
    hidden_dims : list
        Архитектура скрытых слоев VAE
    n_clusters : int
        Количество основных кластеров для KMeans (используется если optimize=False или как центр диапазона)
    n_clusters_range : tuple, optional
        Диапазон количества кластеров для оптимизации (min, max).
        Если None, используется диапазон вокруг n_clusters: (max(2, n_clusters-2), min(n_clusters+3, 15))
    boundary_threshold : float
        Порог для определения boundary точек (0-1)
    max_boundary_clusters : int, optional
        Максимальное количество boundary кластеров.
        Если None, ограничивается количеством основных кластеров (n_clusters).
        Если указано, ограничивается сверху: min(max_boundary_clusters, n_clusters).
    vae_epochs : int
        Количество эпох обучения VAE
    vae_beta : float
        Коэффициент β для β-VAE
    vae_batch_size : int
        Размер батча для обучения VAE
    vae_lr : float
        Learning rate для обучения VAE
    vae_dropout : float
        Вероятность dropout для VAE
    use_batch_norm : bool
        Использовать ли Batch Normalization в VAE
    early_stopping_patience : int
        Количество эпох без улучшения для early stopping
    optimize : bool
        Выполнять ли Bayesian Optimization гиперпараметров
    max_optimization_iterations : int
        Максимальное количество итераций оптимизации
    verbose : bool
        Выводить ли подробную информацию

    Возвращает:
    -----------
    results : dict
        Словарь с результатами
    """
    print("="*80)
    print("ЗАПУСК VAE + KMEANS + BOUNDARY КЛАСТЕРИЗАЦИИ")
    print("="*80)

    # Инициализация времени оптимизации
    optimization_time = 0.0

    # ========== ШАГ 1: Предобработка данных ==========
    if verbose:
        print("\nШаг 1: Предобработка данных...")
    X, preprocessor, feature_info = preprocess_dataframe(df, exclude_columns=exclude_columns)

    # Проверка: latent_dim не должен превышать размерность данных
    max_latent_dim = min(X.shape[1], 25)
    if latent_dim > max_latent_dim:
        if verbose:
            print(f"latent_dim={latent_dim} превышает размерность данных {X.shape[1]}. Устанавливаем latent_dim={max_latent_dim}")
        latent_dim = max_latent_dim

    # ========== ШАГ 1.5: Bayesian Optimization (если включена) ==========
    if optimize:
        if not BAYESIAN_OPT_AVAILABLE:
            if verbose:
                print("\nBayesian Optimization недоступен (scikit-optimize не установлен)")
                print("   Продолжаем с заданными гиперпараметрами...")
            optimize = False
        else:
            if verbose:
                print("\nШаг 1.5: Bayesian Optimization гиперпараметров...")
                print("   Оптимизируются параметры VAE и boundary кластеризации:")
                print("   - VAE: latent_dim, vae_beta")
                print("   - Кластеризация: n_clusters")
                print("   - Boundary detection: boundary_threshold")
                print("   Целевая функция: максимизация silhouette score")

            # Определяем пространство поиска
            max_latent_dim_opt = min(X.shape[1], 25)
            min_latent_dim_opt = min(8, max_latent_dim_opt)

            # Функция для оптимизации (минимизируем отрицательный silhouette score)
            # Оптимизируются параметры всей системы: VAE + кластеризация + boundary detection
            # Используется train/validation split для предотвращения data leakage
            def objective(params):
                opt_latent_dim = int(params[0])
                opt_n_clusters = int(params[1])
                opt_boundary_threshold = params[2]
                opt_vae_beta = params[3]

                try:
                    # Разделяем данные на train и validation для предотвращения data leakage
                    # Используем 80% для обучения, 20% для валидации
                    X_train, X_val = train_test_split(
                        X,
                        test_size=0.2,
                        random_state=RANDOM_STATE,
                        shuffle=True
                    )

                    # Обучение VAE с текущими параметрами на TRAIN данных
                    opt_latent_train, model_temp, _ = train_vae(
                        X_train,
                        hidden_dims=hidden_dims,
                        latent_dim=opt_latent_dim,
                        epochs=min(vae_epochs, 50),  # Меньше эпох для оптимизации
                        beta=opt_vae_beta,
                        batch_size=vae_batch_size,
                        lr=vae_lr,
                        dropout=vae_dropout,
                        use_batch_norm=use_batch_norm,
                        verbose=False
                    )

                    # Кластеризация на TRAIN данных
                    opt_kmeans = KMeans(n_clusters=opt_n_clusters, n_init=5, random_state=RANDOM_STATE)
                    opt_labels_train = opt_kmeans.fit_predict(opt_latent_train)

                    # Получаем латентные представления для VALIDATION данных
                    # Используем обученную модель для получения латентных векторов validation данных
                    model_temp.eval()
                    with torch.no_grad():
                        X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(DEVICE)
                        mu_val, _ = model_temp.encode(X_val_tensor)
                        opt_latent_val = mu_val.cpu().numpy()

                    # Кластеризация на VALIDATION данных (используем центры, обученные на train)
                    opt_labels_val = opt_kmeans.predict(opt_latent_val)

                    # Boundary detection на VALIDATION данных
                    opt_boundary_mask_val, opt_boundary_labels_val = detect_boundary_clusters(
                        opt_latent_val, opt_labels_val, boundary_threshold=opt_boundary_threshold,
                        max_boundary_clusters=max_boundary_clusters
                    )

                    # Метрики вычисляются на VALIDATION данных (opt_latent_val)
                    opt_metrics = evaluate_clustering_metrics(opt_latent_val, opt_labels_val, opt_boundary_labels_val)

                    # Целевая функция: максимизируем silhouette score (минимизируем отрицательный)
                    score = -opt_metrics.get('silhouette', -1.0)

                    return score

                except Exception as e:
                    if verbose:
                        print(f"Ошибка при оптимизации: {e}")
                    return 1.0  # Плохой score при ошибке

            # Определяем диапазон количества кластеров для оптимизации
            if n_clusters_range is not None:
                min_clusters_opt = max(2, int(n_clusters_range[0]))
                max_clusters_opt = min(50, int(n_clusters_range[1]))  # Ограничиваем максимум 50
                if min_clusters_opt >= max_clusters_opt:
                    if verbose:
                        print(f"Внимание: некорректный диапазон кластеров. Используется диапазон вокруг n_clusters={n_clusters}")
                    min_clusters_opt = max(2, n_clusters - 2)
                    max_clusters_opt = min(n_clusters + 3, 15)
            else:
                # Используем диапазон вокруг указанного n_clusters
                min_clusters_opt = max(2, n_clusters - 2)
                max_clusters_opt = min(n_clusters + 3, 15)

            if verbose:
                print(f"   Диапазон количества кластеров для оптимизации: [{min_clusters_opt}, {max_clusters_opt}]")

            # Пространство поиска
            dimensions = [
                Integer(min_latent_dim_opt, max_latent_dim_opt, name='latent_dim'),
                Integer(min_clusters_opt, max_clusters_opt, name='n_clusters'),
                Real(0.3, 0.7, name='boundary_threshold'),
                Real(1.0, 8.0, name='vae_beta')
            ]

            # Запуск оптимизации
            if verbose:
                print(f"   Запуск оптимизации ({max_optimization_iterations} итераций)...")

            optimization_start_time = time.time()
            result = gp_minimize(
                func=objective,
                dimensions=dimensions,
                n_calls=max_optimization_iterations,
                random_state=RANDOM_STATE,
                verbose=verbose
            )
            optimization_time = time.time() - optimization_start_time
            if verbose:
                print(f"   Время оптимизации: {optimization_time:.2f} сек (исключается из общего времени)")

            # Обновляем параметры на оптимальные
            latent_dim = int(result.x[0])
            n_clusters = int(result.x[1])
            boundary_threshold = result.x[2]
            vae_beta = result.x[3]

            if verbose:
                print(f"\nОптимизация завершена!")
                print(f"   Оптимальные параметры:")
                print(f"   - latent_dim: {latent_dim}")
                print(f"   - n_clusters: {n_clusters}")
                print(f"   - boundary_threshold: {boundary_threshold:.3f}")
                print(f"   - vae_beta: {vae_beta:.3f}")
                print(f"   - Лучший score: {-result.fun:.4f}")

    # ========== ШАГ 2: Обучение VAE ==========
    if verbose:
        print("\nШаг 2: Обучение VAE...")
    vae_start_time = time.time()
    latent_vectors, model, history = train_vae(
        X,
        hidden_dims=hidden_dims,
        latent_dim=latent_dim,
        epochs=vae_epochs,
        batch_size=vae_batch_size,
        lr=vae_lr,
        beta=vae_beta,
        dropout=vae_dropout,
        use_batch_norm=use_batch_norm,
        early_stopping_patience=early_stopping_patience,
        verbose=verbose
    )
    vae_time = time.time() - vae_start_time
    if verbose:
        print(f"   Время обучения VAE: {vae_time:.2f} сек")

    # ========== ШАГ 3: Кластеризация KMeans ==========
    if verbose:
        print("\nШаг 3: Кластеризация KMeans...")
    kmeans_start_time = time.time()
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=RANDOM_STATE)
    cluster_labels = kmeans.fit_predict(latent_vectors)
    kmeans_time = time.time() - kmeans_start_time
    if verbose:
        print(f"   Время кластеризации KMeans: {kmeans_time:.2f} сек")

    # ========== ШАГ 4: Обнаружение boundary кластеров ==========
    if verbose:
        print("\nШаг 4: Обнаружение boundary кластеров...")
    boundary_start_time = time.time()
    boundary_mask, boundary_labels = detect_boundary_clusters(
        latent_vectors,
        cluster_labels,
        boundary_threshold=boundary_threshold,
        max_boundary_clusters=max_boundary_clusters
    )
    boundary_time = time.time() - boundary_start_time
    if verbose:
        print(f"   Время обнаружения boundary кластеров: {boundary_time:.2f} сек")

    # Общее время выполнения (VAE + KMeans + Boundary)
    total_execution_time = vae_time + kmeans_time + boundary_time
    if verbose:
        print(f"\n   Общее время выполнения: {total_execution_time:.2f} сек")

    # ========== ШАГ 5: Вычисление метрик ==========
    if verbose:
        print("\nШаг 5: Вычисление метрик...")
    # Метрики вычисляются на латентном пространстве VAE (latent_vectors)
    metrics = evaluate_clustering_metrics(latent_vectors, cluster_labels, boundary_labels)

    # ========== ШАГ 6: Анализ характеристик кластеров ==========
    if verbose:
        print("\nШаг 6: Анализ характеристик кластеров...")
    cluster_descriptions = analyze_cluster_characteristics(
        df, cluster_labels, boundary_labels, preprocessor, feature_info
    )
    cluster_statistics = get_cluster_statistics(
        df, cluster_labels, boundary_labels, feature_info
    )

    # Формируем результаты
    results = {
        'latent_vectors': latent_vectors,
        'cluster_labels': cluster_labels,
        'boundary_labels': boundary_labels,
        'metrics': metrics,
        'model': model,
        'preprocessor': preprocessor,
        'feature_info': feature_info,
        'cluster_descriptions': cluster_descriptions,
        'cluster_statistics': cluster_statistics,
        'X_processed': X,
        'history': history,
        'execution_time': total_execution_time,
        'vae_time': vae_time,
        'kmeans_time': kmeans_time,
        'boundary_time': boundary_time,
        'optimization_time': optimization_time
    }

    if verbose:
        print("\nКластеризация завершена!")
        print(f"   - Найдено основных кластеров: {len(np.unique(cluster_labels))}")
        print(f"   - Найдено boundary кластеров: {len(np.unique(boundary_labels[boundary_labels != -1]))}")
        print(f"   - Boundary точек: {boundary_mask.sum()} ({boundary_mask.sum()/len(df)*100:.2f}%)")

    return results

## Статистическая методология сравнения методов

Для доказательства статистической значимости различий между методами используется статистический подход:

### 1. Тест Фридмана (Friedman Test)

**Назначение**: проверяет, есть ли статистически значимые различия между методами в целом.

**Применение**:
- Используется для связанных выборок (все методы работают на одних и тех же данных)
- Непараметрический тест (не требует нормальности распределения)
- Нулевая гипотеза: все методы эквивалентны

### 2. Post-hoc тесты (Wilcoxon Signed-Rank Test)

**Назначение**: попарные сравнения методов после обнаружения общих различий.

### 3. Коррекция на множественные сравнения

**Проблема**: при множественных попарных сравнениях возрастает вероятность ошибки I рода (ложные положительные результаты).

**Решение**: применение коррекции Bonferroni (или FDR):
- Корректирует p-values с учетом количества сравнений
- Обеспечивает контроль уровня значимости на уровне 0.05

### 4. Интерпретация результатов

- **p-value < 0.05**: статистически значимые различия между методами
- **NaN p-value**: статистический тест невозможен (недостаточно данных, отсутствие вариативности, или идентичные результаты)
- Тепловые карты p-values визуализируют значимость различий между всеми парами методов


In [ ]:
# @title
def plot_training_history(results, figsize=(16, 5)):
    """
    Визуализирует историю обучения VAE модели.

    Параметры:
    -----------
    results : dict
        Результаты функции run_vae_boundary_clustering (должен содержать 'history')
    figsize : tuple
        Размер фигуры

    Возвращает:
    -----------
    fig : matplotlib.figure.Figure
        Объект фигуры
    """
    if 'history' not in results:
        print("Внимание: история обучения недоступна в results")
        return None

    history = results['history']

    if not history or len(history.get('loss', [])) == 0:
        print("Внимание: история обучения пуста")
        return None

    fig, axes = plt.subplots(1, 3, figsize=figsize)

    epochs = range(1, len(history['loss']) + 1)

    # 1. Общий Loss
    ax = axes[0]
    ax.plot(epochs, history['loss'], 'b-', linewidth=2, label='Total Loss')
    ax.set_xlabel('Эпоха', fontsize=12)
    ax.set_ylabel('Loss', fontsize=12)
    ax.set_title('Общий Loss', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # 2. Reconstruction Loss и KL Loss
    ax = axes[1]
    ax.plot(epochs, history['recon_loss'], 'g-', linewidth=2, label='Reconstruction Loss')
    ax.plot(epochs, history['kl_loss'], 'r-', linewidth=2, label='KL Loss')
    ax.set_xlabel('Эпоха', fontsize=12)
    ax.set_ylabel('Loss', fontsize=12)
    ax.set_title('Reconstruction и KL Loss', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # 3. Отношение KL Loss к Reconstruction Loss
    ax = axes[2]
    if len(history['recon_loss']) > 0 and np.mean(history['recon_loss']) > 0:
        kl_recon_ratio = np.array(history['kl_loss']) / (np.array(history['recon_loss']) + 1e-10)
        ax.plot(epochs, kl_recon_ratio, 'purple', linewidth=2, label='KL / Recon Ratio')
        ax.set_xlabel('Эпоха', fontsize=12)
        ax.set_ylabel('Отношение', fontsize=12)
        ax.set_title('Отношение KL Loss к Reconstruction Loss', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'Недостаточно данных\nдля визуализации',
               ha='center', va='center', fontsize=12, transform=ax.transAxes)

    plt.suptitle('История обучения VAE модели', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    return fig

print("Функция визуализации истории обучения определена")


Функция визуализации истории обучения определена


In [ ]:
# @title
def visualize_clusters(results, method_name="VAE + KMeans + Boundary"):
    """
    Визуализирует результаты кластеризации в 2D и 3D.

    Параметры:
    -----------
    results : dict
        Результаты функции run_vae_boundary_clustering
    method_name : str
        Название метода для отображения
    """
    latent_vectors = results['latent_vectors']
    cluster_labels = results['cluster_labels']
    boundary_labels = results['boundary_labels']

    # Применяем PCA для визуализации
    pca_2d = PCA(n_components=2, random_state=RANDOM_STATE)
    vectors_2d = pca_2d.fit_transform(latent_vectors)

    pca_3d = PCA(n_components=3, random_state=RANDOM_STATE)
    vectors_3d = pca_3d.fit_transform(latent_vectors)

    unique_main = np.unique(cluster_labels[boundary_labels == -1])
    unique_boundary = np.unique(boundary_labels[boundary_labels != -1])

    main_colors = plt.cm.tab10(np.linspace(0, 1, len(unique_main)))
    color_map = {cid: main_colors[i] for i, cid in enumerate(unique_main)}

    if len(unique_boundary) > 0:
        boundary_colors = plt.cm.Reds(np.linspace(0.4, 0.9, len(unique_boundary)))
        for i, cid in enumerate(unique_boundary):
            color_map[cid] = boundary_colors[i]

    # 2D и 3D визуализация
    fig = plt.figure(figsize=(20, 8))

    # 2D
    ax1 = fig.add_subplot(121)
    for cid in unique_main:
        mask = (cluster_labels == cid) & (boundary_labels == -1)
        if mask.sum() > 0:
            ax1.scatter(vectors_2d[mask, 0], vectors_2d[mask, 1],
                       c=[color_map[cid]], label=f'Cluster {cid}',
                       alpha=0.6, s=15, edgecolors='black', linewidths=0.3)

    for cid in unique_boundary:
        mask = boundary_labels == cid
        if mask.sum() > 0:
            ax1.scatter(vectors_2d[mask, 0], vectors_2d[mask, 1],
                       c=[color_map[cid]], label=f'Boundary {cid}',
                       alpha=0.8, s=25, edgecolors='darkred', linewidths=1.0, marker='^')

    ax1.set_xlabel('PC1', fontsize=12)
    ax1.set_ylabel('PC2', fontsize=12)
    ax1.set_title(f'{method_name} - 2D Visualization', fontsize=14, fontweight='bold')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax1.grid(True, alpha=0.3)

    # 3D
    ax2 = fig.add_subplot(122, projection='3d')
    for cid in unique_main:
        mask = (cluster_labels == cid) & (boundary_labels == -1)
        if mask.sum() > 0:
            ax2.scatter(vectors_3d[mask, 0], vectors_3d[mask, 1], vectors_3d[mask, 2],
                       c=[color_map[cid]], label=f'Cluster {cid}',
                       alpha=0.6, s=15, edgecolors='black', linewidths=0.3)

    for cid in unique_boundary:
        mask = boundary_labels == cid
        if mask.sum() > 0:
            ax2.scatter(vectors_3d[mask, 0], vectors_3d[mask, 1], vectors_3d[mask, 2],
                       c=[color_map[cid]], label=f'Boundary {cid}',
                       alpha=0.8, s=25, edgecolors='darkred', linewidths=1.0, marker='^')

    ax2.set_xlabel('PC1', fontsize=12)
    ax2.set_ylabel('PC2', fontsize=12)
    ax2.set_zlabel('PC3', fontsize=12)
    ax2.set_title(f'{method_name} - 3D Visualization', fontsize=14, fontweight='bold')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)

    plt.tight_layout()
    plt.show()

    return fig


In [ ]:
# @title
def print_technical_metrics(results, comparison_df=None):
    """
    Выводит технические метрики (время выполнения) для всех методов.

    Параметры:
    -----------
    results : dict
        Результаты VAE + KMeans + Boundary метода
    comparison_df : pd.DataFrame, optional
        DataFrame с метриками всех методов (если доступен)
    """
    print("\n" + "="*80)
    print("ТЕХНИЧЕСКИЕ МЕТРИКИ")
    print("="*80)

    # Метрики нашего метода
    if 'execution_time' in results:
        print(f"\nVAE + KMeans + Boundary метод:")
        print(f"   Время выполнения алгоритма: {results['execution_time']:.2f} сек")
        if 'vae_time' in results:
            print(f"      - Обучение VAE: {results['vae_time']:.2f} сек ({results['vae_time']/results['execution_time']*100:.1f}%)")
        if 'kmeans_time' in results:
            print(f"      - Кластеризация KMeans: {results['kmeans_time']:.2f} сек ({results['kmeans_time']/results['execution_time']*100:.1f}%)")
        if 'boundary_time' in results:
            print(f"      - Обнаружение boundary: {results['boundary_time']:.2f} сек ({results['boundary_time']/results['execution_time']*100:.1f}%)")

        # Время оптимизации
        if 'optimization_time' in results and results['optimization_time'] > 0:
            total_with_opt = results['execution_time'] + results['optimization_time']
            print(f"\n   Время оптимизации гиперпараметров: {results['optimization_time']:.2f} сек")
            print(f"   Общее время (с оптимизацией): {total_with_opt:.2f} сек")
            print(f"   Примечание: время оптимизации исключено из метрик времени выполнения")

    # Метрики baseline методов
    if comparison_df is not None and 'execution_time' in comparison_df.columns:
        print(f"\nBaseline методы:")
        for _, row in comparison_df.iterrows():
            method = row['method']
            time_val = row['execution_time']
            if pd.notna(time_val):
                if time_val > 60:
                    time_str = f"{time_val/60:.1f} мин"
                else:
                    time_str = f"{time_val:.2f} сек"
                print(f"   - {method:25s}: {time_str}")

        # Сравнение
        if 'execution_time' in results and 'execution_time' in comparison_df.columns:
            vae_time = results['execution_time']
            # Исключаем VAE методы из сравнения baseline
            baseline_times = comparison_df[
                (~comparison_df['method'].str.contains('VAE', na=False))
            ]['execution_time'].dropna()
            if len(baseline_times) > 0:
                avg_baseline_time = baseline_times.mean()
                min_baseline_time = baseline_times.min()
                max_baseline_time = baseline_times.max()

                print(f"\nСравнение:")
                print(f"   - Среднее время baseline методов: {avg_baseline_time:.2f} сек")
                print(f"   - Наш метод: {vae_time:.2f} сек")
                if vae_time > 0:
                    ratio = vae_time / avg_baseline_time
                    print(f"   - Отношение: {ratio:.2f}x {'(медленнее)' if ratio > 1 else '(быстрее)'}")

print("Функция вывода технических метрик определена")


Функция вывода технических метрик определена


## Сравнение с базовыми методами и статистический анализ

Сравнение нашего метода (VAE + KMeans + Boundary) с классическими методами кластеризации и проверка статистической значимости различий.


In [ ]:
# @title
def compare_with_baselines(X, results, n_runs=10, vae_params=None):
    """
    Сравнивает VAE + KMeans + Boundary метод с baseline методами.

    Выполняет несколько запусков для статистической значимости.

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные (можно получить из results['preprocessor'].transform(df))
        или передать напрямую предобработанные данные
    results : dict
        Результаты VAE + KMeans + Boundary метода. Должен содержать 'X_processed' для переобучения VAE.
    n_runs : int
        Количество запусков для каждого метода (для статистики)
    vae_params : dict, optional
        Параметры для обучения VAE. Если None, используются параметры из results или значения по умолчанию.
        Используются предобработанные данные из results['X_processed'] для гарантии одинаковой предобработки.

    Возвращает:
    -----------
    comparison_df : pd.DataFrame
        DataFrame с метриками всех методов
    all_metrics_runs : dict
        Словарь со всеми запусками для статистического анализа
    """



    all_metrics_runs = {
        'VAE + KMeans + Boundary': [],
        'VAE Latent Space': [],
        'PCA + KMeans': [],
        'Standard KMeans': [],
        'DBSCAN': [],
        'Agglomerative': []
    }

    # Используем результаты VAE метода для определения количества кластеров
    vae_latent = results['latent_vectors']
    vae_clusters = results['cluster_labels']
    vae_boundary = results['boundary_labels']

    # Определяем количество кластеров из результатов VAE
    n_clusters_vae = len(np.unique(vae_clusters))

    # Используем предобработанные данные из results для переобучения VAE
    if 'X_processed' in results and results['X_processed'] is not None:

        # Определяем параметры для VAE из results или используем значения по умолчанию
        if vae_params is None:
            vae_params = {
                'latent_dim': results.get('latent_dim', 15),
                'n_clusters': n_clusters_vae,
                'boundary_threshold': results.get('boundary_threshold', 0.5),
                'vae_epochs': results.get('vae_epochs', 150),
                'vae_beta': results.get('vae_beta', 4.0),
                'optimize': False,  # Отключаем оптимизацию для ускорения
                'verbose': False  # Отключаем подробный вывод для ускорения
            }

        X_processed = results['X_processed']

        # Запускаем VAE метод несколько раз с переобучением
        global RANDOM_STATE
        original_random_state = RANDOM_STATE

        for run in range(n_runs):
            # Устанавливаем random_state для разнообразия
            current_random_state = original_random_state + run
            RANDOM_STATE = current_random_state
            np.random.seed(current_random_state)
            torch.manual_seed(current_random_state)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(current_random_state)

            if run % 2 == 0 or run == n_runs - 1:
                print(f"  VAE метод: Запуск {run+1}/{n_runs}...")

            try:

                # Обучение VAE
                latent_vectors, model, history = train_vae(
                    X_processed,
                    hidden_dims=vae_params.get('hidden_dims', [512, 256, 128]),
                    latent_dim=vae_params['latent_dim'],
                    epochs=vae_params['vae_epochs'],
                    batch_size=vae_params.get('vae_batch_size', 256),
                    lr=vae_params.get('vae_lr', 1e-4),
                    beta=vae_params['vae_beta'],
                    dropout=vae_params.get('vae_dropout', 0.2),
                    use_batch_norm=vae_params.get('use_batch_norm', True),
                    early_stopping_patience=vae_params.get('early_stopping_patience', 20),
                    verbose=False
                )

                # Кластеризация KMeans
                kmeans = KMeans(n_clusters=vae_params['n_clusters'], n_init=10, random_state=current_random_state)
                cluster_labels = kmeans.fit_predict(latent_vectors)

                # Обнаружение boundary кластеров
                boundary_mask, boundary_labels = detect_boundary_clusters(
                    latent_vectors,
                    cluster_labels,
                    boundary_threshold=vae_params['boundary_threshold'],
                    weight_distance=vae_params.get('weight_distance', 0.5),
                    weight_entropy=vae_params.get('weight_entropy', 0.5)
                )

                # Вычисление метрик
                metrics = evaluate_clustering_metrics(latent_vectors, cluster_labels, boundary_labels)

                # Добавляем результаты VAE + KMeans + Boundary
                vae_metrics = metrics.copy()
                vae_metrics['method'] = 'VAE + KMeans + Boundary'
                all_metrics_runs['VAE + KMeans + Boundary'].append(vae_metrics)

                # Добавляем результаты VAE Latent Space
                vae_latent_no_boundary = np.full(len(boundary_labels), -1)
                vae_latent_metrics = evaluate_clustering_metrics(
                    latent_vectors,
                    cluster_labels,
                    vae_latent_no_boundary
                )
                vae_latent_metrics['method'] = 'VAE Latent Space'
                all_metrics_runs['VAE Latent Space'].append(vae_latent_metrics)

            except Exception as e:
                # Продолжаем с другими запусками
                pass

        # Восстанавливаем исходный random_state
        RANDOM_STATE = original_random_state
        np.random.seed(original_random_state)
        torch.manual_seed(original_random_state)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(original_random_state)
    else:

        vae_metrics = results['metrics'].copy()
        vae_metrics['method'] = 'VAE + KMeans + Boundary'
        if 'execution_time' in results:
            vae_metrics['execution_time'] = results['execution_time']
        all_metrics_runs['VAE + KMeans + Boundary'].append(vae_metrics)

        vae_latent_no_boundary = np.full(len(results['boundary_labels']), -1)
        vae_latent_metrics = evaluate_clustering_metrics(
            results['latent_vectors'],
            results['cluster_labels'],
            vae_latent_no_boundary
        )
        vae_latent_metrics['method'] = 'VAE Latent Space'
        if 'vae_time' in results:
            vae_latent_metrics['execution_time'] = results['vae_time']
        elif 'execution_time' in results:
            vae_latent_metrics['execution_time'] = results['execution_time'] * 0.9
        else:
            vae_latent_metrics['execution_time'] = 0.0
        all_metrics_runs['VAE Latent Space'].append(vae_latent_metrics)

    for run in range(n_runs):
        np.random.seed(RANDOM_STATE + run)
        if run % 2 == 0 or run == n_runs - 1:
            print(f"  Запуск {run+1}/{n_runs}...")

        # 3. PCA + KMeans
        start_time = time.time()
        pca = PCA(n_components=min(15, X.shape[1]), random_state=RANDOM_STATE + run)
        X_pca = pca.fit_transform(X)
        kmeans_pca = KMeans(n_clusters=n_clusters_vae, n_init=10,
                           random_state=RANDOM_STATE + run)
        labels_pca = kmeans_pca.fit_predict(X_pca)
        # Не применяем boundary detection к baseline методам
        boundary_labels_pca = np.full(len(labels_pca), -1)
        pca_time = time.time() - start_time
        # Метрики вычисляются на пространстве PCA (X_pca)
        metrics_pca = evaluate_clustering_metrics(X_pca, labels_pca, boundary_labels_pca)
        metrics_pca['method'] = 'PCA + KMeans'
        metrics_pca['execution_time'] = pca_time
        all_metrics_runs['PCA + KMeans'].append(metrics_pca)

        # 4. Standard KMeans
        start_time = time.time()
        kmeans_std = KMeans(n_clusters=n_clusters_vae, n_init=10,
                          random_state=RANDOM_STATE + run)
        labels_std = kmeans_std.fit_predict(X)
        # Не применяем boundary detection к baseline методам
        boundary_labels_std = np.full(len(labels_std), -1)
        kmeans_time = time.time() - start_time
        # Метрики вычисляются на исходном пространстве (X)
        metrics_std = evaluate_clustering_metrics(X, labels_std, boundary_labels_std)
        metrics_std['method'] = 'Standard KMeans'
        metrics_std['execution_time'] = kmeans_time
        all_metrics_runs['Standard KMeans'].append(metrics_std)

        # 5. DBSCAN
        start_time = time.time()
        nbrs = NearestNeighbors(n_neighbors=4).fit(X)
        distances, _ = nbrs.kneighbors(X)
        k_distances = np.sort(distances[:, -1])
        eps = np.percentile(k_distances, 95)
        dbscan = DBSCAN(eps=eps, min_samples=30)
        labels_dbscan = dbscan.fit_predict(X)
        dbscan_time = time.time() - start_time

        # Для DBSCAN boundary detection не применяем
        boundary_labels_dbscan = np.full(len(labels_dbscan), -1)
        # Метрики вычисляются на исходном пространстве (X)
        metrics_dbscan = evaluate_clustering_metrics(X, labels_dbscan, boundary_labels_dbscan)
        metrics_dbscan['method'] = 'DBSCAN'
        metrics_dbscan['execution_time'] = dbscan_time
        all_metrics_runs['DBSCAN'].append(metrics_dbscan)

        # 6. Agglomerative Clustering
        start_time = time.time()
        agg = AgglomerativeClustering(n_clusters=n_clusters_vae, linkage='ward')
        labels_agg = agg.fit_predict(X)
        # Не применяем boundary detection к baseline методам
        boundary_labels_agg = np.full(len(labels_agg), -1)
        agg_time = time.time() - start_time
        # Метрики вычисляются на исходном пространстве (X)
        metrics_agg = evaluate_clustering_metrics(X, labels_agg, boundary_labels_agg)
        metrics_agg['method'] = 'Agglomerative'
        metrics_agg['execution_time'] = agg_time
        all_metrics_runs['Agglomerative'].append(metrics_agg)

    # Создаем DataFrame со средними значениями
    comparison_data = []
    for method, metrics_list in all_metrics_runs.items():
        # Вычисляем средние значения метрик
        avg_metrics = {}
        for key in metrics_list[0].keys():
            if key != 'method':
                values = [m[key] for m in metrics_list if key in m]
                if values:
                    avg_metrics[key] = np.mean(values)
        avg_metrics['method'] = method
        comparison_data.append(avg_metrics)

    comparison_df = pd.DataFrame(comparison_data)



    return comparison_df, all_metrics_runs


In [ ]:
# @title
def statistical_comparison(all_metrics_runs, metrics_to_compare=None, correction_method='bonferroni'):
    """
    Выполняет статистическое сравнение методов с помощью теста Фридмана и post-hoc тестов.

    Правильный статистический подход для сравнения нескольких методов кластеризации:
    1. Тест Фридмана (Friedman test) - проверяет, есть ли различия между методами в целом
    2. Post-hoc тесты (Wilcoxon signed-rank) - попарные сравнения для связанных выборок
    3. Коррекция на множественные сравнения (Bonferroni или FDR) - уменьшает вероятность ошибки I рода

    Почему не Mann-Whitney U test:
    - Mann-Whitney U предназначен для независимых выборок
    - У нас связанные выборки (все методы работают на одних и тех же данных)
    - Множественные сравнения требуют коррекции

    Параметры:
    -----------
    all_metrics_runs : dict
        Словарь со всеми запусками методов (из compare_with_baselines)
    metrics_to_compare : list, optional
        Список метрик для сравнения. Если None, используются основные метрики.
    correction_method : str, optional
        Метод коррекции на множественные сравнения: 'bonferroni', 'fdr_bh', 'fdr_by', 'holm'
        По умолчанию 'bonferroni' (самый консервативный)

    Возвращает:
    -----------
    pvalue_matrices : dict
        Словарь с матрицами p-values для каждой метрики (с коррекцией)
        Примечание: NaN значения в матрице означают, что статистический тест
        невозможен для данной пары методов (недостаточно данных, отсутствие
        вариативности, или идентичные результаты)
    friedman_results : dict
        Словарь с результатами теста Фридмана для каждой метрики
    """
    if metrics_to_compare is None:
        # Основные метрики для сравнения
        metrics_to_compare = [
            'silhouette',
            'davies_bouldin',
            'calinski_harabasz',
            'boundary_ratio',
            'boundary_density',
            'boundary_stability'
        ]

    methods = list(all_metrics_runs.keys())

    # Создаем матрицы p-values для каждой метрики
    pvalue_matrices = {}
    friedman_results = {}

    print("="*80)
    print("СТАТИСТИЧЕСКОЕ СРАВНЕНИЕ МЕТОДОВ")
    print("Тест Фридмана + Wilcoxon signed-rank (post-hoc) с коррекцией на множественные сравнения")
    print("="*80)

    if not MULTIPLE_TESTING_AVAILABLE:
        correction_method = None

    for metric in metrics_to_compare:
        available_methods = []
        for method in methods:
            if len(all_metrics_runs[method]) > 0 and metric in all_metrics_runs[method][0]:
                if len(all_metrics_runs[method]) >= 2:
                    available_methods.append(method)
                elif len(all_metrics_runs[method]) == 1:
                    pass

        if len(available_methods) < 2:
            continue

        # Создаем матрицу p-values
        pvalue_matrix = pd.DataFrame(
            index=available_methods,
            columns=available_methods,
            dtype=float
        )

        # Шаг 1: Подготовка данных для теста Фридмана
        # Преобразуем данные в матрицу: строки = запуски, столбцы = методы
        data_matrix = []
        valid_runs = None

        for method in available_methods:
            values = [m[metric] for m in all_metrics_runs[method]
                     if metric in m and not (np.isnan(m[metric]) or np.isinf(m[metric]))]
            if valid_runs is None:
                valid_runs = len(values)
            elif len(values) != valid_runs:
                # Если количество запусков различается, используем минимальное
                valid_runs = min(valid_runs, len(values))
            data_matrix.append(values[:valid_runs] if len(values) >= valid_runs else values)

        # Транспонируем: строки = запуски, столбцы = методы
        data_matrix = np.array(data_matrix).T  # shape: (n_runs, n_methods)

        if data_matrix.shape[0] < 2 or data_matrix.shape[1] < 2:
            continue

        # Шаг 2: Тест Фридмана
        try:
            friedman_stat, friedman_pvalue = friedmanchisquare(*data_matrix.T)
            friedman_results[metric] = {
                'statistic': friedman_stat,
                'pvalue': friedman_pvalue,
                'significant': friedman_pvalue < 0.05
            }
        except Exception as e:
            friedman_results[metric] = {'statistic': np.nan, 'pvalue': np.nan, 'significant': False}

        # Шаг 3: Post-hoc попарные сравнения (Wilcoxon signed-rank test для связанных выборок)
        # Собираем все p-values для коррекции
        pairwise_pvalues = []
        pairwise_pairs = []

        for i, method1 in enumerate(available_methods):
            for j, method2 in enumerate(available_methods):
                if i == j:
                    pvalue_matrix.loc[method1, method2] = 1.0  # Сравнение с самим собой
                elif j > i:  # Вычисляем только для верхнего треугольника
                    # Извлекаем значения метрики для обоих методов (связанные выборки)
                    values1 = data_matrix[:, i]  # Значения для method1 по всем запускам
                    values2 = data_matrix[:, j]  # Значения для method2 по всем запускам

                    # Удаляем NaN и Inf значения
                    mask = ~(np.isnan(values1) | np.isnan(values2) | np.isinf(values1) | np.isinf(values2))
                    values1_clean = values1[mask]
                    values2_clean = values2[mask]

                    if len(values1_clean) < 2:
                        pvalue = np.nan
                    else:
                        # Проверяем вариативность
                        diff = values1_clean - values2_clean
                        std_diff = np.std(diff)

                        if std_diff == 0:

                            if abs(np.mean(diff)) < 1e-10:
                                pvalue = 1.0
                            else:
                                pvalue = np.nan

                        else:
                            # Wilcoxon signed-rank test для связанных выборок
                            try:
                                # Проверяем вариативность перед тестом
                                if len(values1_clean) != len(values2_clean):
                                    pvalue = np.nan
                                elif len(values1_clean) < 2:
                                    pvalue = np.nan
                                else:
                                    # Вычисляем разности для проверки
                                    diff = values1_clean - values2_clean
                                    if np.std(diff) < 1e-10:
                                        # Нет различий между методами
                                        pvalue = 1.0
                                    else:
                                        statistic, pvalue = wilcoxon(values1_clean, values2_clean, alternative='two-sided')
                            except ValueError as e:
                                if "identical" in str(e).lower() or "zero" in str(e).lower():
                                    pvalue = 1.0
                                else:
                                    pvalue = np.nan
                            except Exception as e:
                                pvalue = np.nan

                    pairwise_pvalues.append(pvalue)
                    pairwise_pairs.append((i, j, method1, method2))

        # Шаг 4: Коррекция на множественные сравнения
        correction_applied = False

        if correction_method and MULTIPLE_TESTING_AVAILABLE and len(pairwise_pvalues) > 0:
            valid_indices = [idx for idx, p in enumerate(pairwise_pvalues) if not np.isnan(p)]
            if len(valid_indices) > 0:
                valid_pvalues = [pairwise_pvalues[idx] for idx in valid_indices]
                try:
                    _, corrected_pvalues, _, _ = multipletests(
                        valid_pvalues,
                        alpha=0.05,
                        method=correction_method
                    )


                    # Заменяем скорректированные p-values
                    for idx, corrected_p in zip(valid_indices, corrected_pvalues):
                        pairwise_pvalues[idx] = corrected_p
                    correction_applied = True
                except Exception as e:
                    pass

        n_nan_pvalues = 0
        for (i, j, method1, method2), pvalue in zip(pairwise_pairs, pairwise_pvalues):
            if np.isnan(pvalue):
                n_nan_pvalues += 1
                pvalue_matrix.loc[method1, method2] = np.nan
                pvalue_matrix.loc[method2, method1] = np.nan
            else:
                pvalue_matrix.loc[method1, method2] = pvalue
                pvalue_matrix.loc[method2, method1] = pvalue

        pvalue_matrices[metric] = pvalue_matrix



    return pvalue_matrices, friedman_results


In [ ]:
# @title
def plot_pvalue_heatmap(pvalue_matrices, significance_level=0.05, figsize=(18, 5)):
    """
    Создает 3 тепловые карты p-values для метрик: silhouette, davies_bouldin, calinski_harabasz.

    Параметры:
    -----------
    pvalue_matrices : dict
        Словарь с матрицами p-values для каждой метрики
    significance_level : float
        Уровень значимости (обычно 0.05)
    figsize : tuple
        Размер фигуры

    Возвращает:
    -----------
    fig : matplotlib.figure.Figure
        Объект фигуры
    """
    # Метрики для отображения
    metrics_to_plot = ['silhouette', 'davies_bouldin', 'calinski_harabasz']

    # Проверяем, какие метрики доступны
    available_metrics = [m for m in metrics_to_plot if m in pvalue_matrices]

    if len(available_metrics) == 0:
        raise ValueError("Нет доступных матриц p-values для отображения")


    fig, axes = plt.subplots(1, 3, figsize=figsize)

    # Если метрик меньше 3, скрываем лишние subplots
    for idx in range(len(available_metrics), 3):
        axes[idx].axis('off')

    # Создаем тепловую карту для каждой метрики
    for idx, metric in enumerate(available_metrics):
        ax = axes[idx]
        pvalue_matrix = pvalue_matrices[metric].copy()

        n_nan_in_matrix = 0

        if pvalue_matrix.empty or pvalue_matrix.isna().all().all():
            print(f"   Матрица для метрики '{metric}' пустая или содержит только NaN. Пропускаем.")
            ax.axis('off')
            ax.text(0.5, 0.5, f'Нет данных\nдля {metric}',
                   ha='center', va='center', fontsize=12)
            continue

        pvalue_matrix = pvalue_matrix.clip(lower=0.0)

        annotations = pvalue_matrix.copy().astype(str)
        n_nan_in_matrix = 0
        for i in range(len(pvalue_matrix)):
            for j in range(len(pvalue_matrix.columns)):
                pval = pvalue_matrix.iloc[i, j]
                if pd.isna(pval):
                    n_nan_in_matrix += 1
                    if i == j:
                        annotations.iloc[i, j] = '-'
                    else:
                        annotations.iloc[i, j] = 'N/A'
                elif i == j:
                    annotations.iloc[i, j] = '-'
                else:
                    # Показываем только "<0.05" или ">=0.05"
                    if pval < significance_level:
                        annotations.iloc[i, j] = '<0.05'
                    else:
                        annotations.iloc[i, j] = '>=0.05'



        from matplotlib.colors import ListedColormap, BoundaryNorm

        if pvalue_matrix.empty or pvalue_matrix.isna().all().all():
            max_pval = 1.0
        else:
            max_pval = np.nanmax(pvalue_matrix.values) if pvalue_matrix.size > 0 else 1.0
            if np.isnan(max_pval) or np.isinf(max_pval):
                max_pval = 1.0
        max_display = max(1.0, max_pval * 1.1)
        bounds = [0.0, significance_level, 1.0, max_display]
        # 3 цвета: зеленый для значимых, серый для незначимых, белый для диагонали
        colors_list = ['#27ae60', '#95a5a6', '#ecf0f1']
        cmap_custom = ListedColormap(colors_list)
        # BoundaryNorm: для каждого интервала между границами используется соответствующий цвет
        # Интервал [0, 0.05) -> цвет 0 (зеленый)
        # Интервал [0.05, 1.0) -> цвет 1 (серый)
        # Интервал [1.0, max_display] -> цвет 1 (серый, для значений > 1.0 после коррекции)
        norm = BoundaryNorm(bounds, len(colors_list), clip=True)

        sns.heatmap(
            pvalue_matrix,
            annot=annotations,
            fmt='',
            cmap=cmap_custom,
            norm=norm,
            vmin=0,
            vmax=max_display,
            square=True,
            linewidths=1.0,
            cbar_kws={'label': 'p-value', 'ticks': [0, significance_level, 1.0]},
            annot_kws={'size': 8},
            ax=ax,
            xticklabels=True,
            yticklabels=True,
            cbar=True
        )

        metric_names = {
            'silhouette': 'Silhouette Score',
            'davies_bouldin': 'Davies-Bouldin Index',
            'calinski_harabasz': 'Calinski-Harabasz Score'
        }
        title_suffix = f'(зеленый = значимые различия, p < {significance_level}'
        if n_nan_in_matrix > 0:
            title_suffix += f'; N/A = тест невозможен'
        title_suffix += ')'
        ax.set_title(f'{metric_names.get(metric, metric)}\n{title_suffix}',
                    fontsize=12, fontweight='bold', pad=10)
        ax.set_xlabel('Метод 1', fontsize=10)
        ax.set_ylabel('Метод 2', fontsize=10)

        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=9)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=9)

    plt.suptitle('Тепловые карты p-values между методами (статистическая значимость различий)',
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    return fig


## Анализ чувствительности к параметрам

Для понимания устойчивости алгоритма важно анализировать влияние гиперпараметров на результаты.

### Анализ чувствительности к весам метрик

Веса `weight_distance` и `weight_entropy` в комбинированной boundary метрике влияют на:
- Количество обнаруженных boundary точек
- Качество boundary кластеров (плотность, выравненность)
- Общее качество кластеризации (silhouette score)

**Методология анализа**:
- Тестирование сетки значений весов (от 0.1 до 0.9)
- Вычисление метрик для каждой комбинации
- Визуализация зависимости метрик от весов
- Определение оптимальной комбинации весов

Это позволяет:
1. Понять, какие веса дают лучшие результаты
2. Оценить устойчивость алгоритма к изменению весов
3. Найти баланс между метрикой расстояний и энтропии


In [ ]:
# @title
def analyze_weight_sensitivity(latent_vectors, cluster_labels, X_original=None,
                               weight_combinations=None, boundary_threshold=0.5,
                               metrics_to_evaluate=None):
    """
    Анализирует чувствительность boundary detection к весам метрик.

    Параметры:
    -----------
    latent_vectors : np.ndarray
        Латентные представления данных
    cluster_labels : np.ndarray
        Метки основных кластеров
    X_original : np.ndarray, optional
        Исходные данные (используется только для анализа характеристик кластеров, если нужен)
        Метрики кластеризации вычисляются на латентном пространстве (latent_vectors)
    weight_combinations : list, optional
        Список кортежей (weight_distance, weight_entropy).
        Если None, используется сетка значений от 0.1 до 0.9 с шагом 0.1
    boundary_threshold : float
        Порог для определения boundary точек
    metrics_to_evaluate : list, optional
        Список метрик для оценки. Если None, используются все метрики.

    Возвращает:
    -----------
    results_df : pd.DataFrame
        DataFrame с результатами для каждой комбинации весов
    """
    if weight_combinations is None:
        # Создаем сетку весов: weight_distance от 0.1 до 0.9, weight_entropy = 1 - weight_distance
        weight_combinations = [(w, 1.0 - w) for w in np.arange(0.1, 1.0, 0.1)]

    if metrics_to_evaluate is None:
        metrics_to_evaluate = ['silhouette', 'davies_bouldin', 'calinski_harabasz',
                              'boundary_ratio', 'boundary_density', 'boundary_alignment', 'boundary_stability']

    results = []

    print("="*80)
    print("АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ К ВЕСАМ МЕТРИК")
    print("="*80)
    print(f"Тестируем {len(weight_combinations)} комбинаций весов...")

    for idx, (w_dist, w_ent) in enumerate(weight_combinations):
        if idx % 5 == 0 or idx == len(weight_combinations) - 1:
            print(f"  Комбинация {idx+1}/{len(weight_combinations)}: weight_distance={w_dist:.2f}, weight_entropy={w_ent:.2f}")

        # Обнаружение boundary кластеров с текущими весами
        boundary_mask, boundary_labels = detect_boundary_clusters(
            latent_vectors, cluster_labels,
            boundary_threshold=boundary_threshold,
            weight_distance=w_dist,
            weight_entropy=w_ent
        )

        # Метрики вычисляются на латентном пространстве (latent_vectors)
        metrics = evaluate_clustering_metrics(latent_vectors, cluster_labels, boundary_labels)

        # Сохраняем результаты
        result = {
            'weight_distance': w_dist,
            'weight_entropy': w_ent,
            'boundary_points': boundary_mask.sum(),
            'boundary_ratio': boundary_mask.sum() / len(cluster_labels) * 100
        }

        # Добавляем метрики
        for metric in metrics_to_evaluate:
            if metric in metrics:
                result[metric] = metrics[metric]
            else:
                result[metric] = np.nan

        results.append(result)

    results_df = pd.DataFrame(results)

    print("\nАнализ завершен!")
    print(f"Лучшая комбинация по silhouette score:")
    best_idx = results_df['silhouette'].idxmax()
    best_row = results_df.loc[best_idx]
    print(f"   weight_distance={best_row['weight_distance']:.2f}, weight_entropy={best_row['weight_entropy']:.2f}")
    print(f"   silhouette={best_row['silhouette']:.4f}, boundary_ratio={best_row['boundary_ratio']:.2f}%")

    return results_df


def visualize_weight_sensitivity(results_df, figsize=(16, 10)):
    """
    Визуализирует результаты анализа чувствительности к весам.

    Параметры:
    -----------
    results_df : pd.DataFrame
        Результаты из analyze_weight_sensitivity
    figsize : tuple
        Размер фигуры
    """
    fig, axes = plt.subplots(2, 3, figsize=figsize)
    axes = axes.flatten()

    # 1. Silhouette Score
    ax = axes[0]
    ax.plot(results_df['weight_distance'], results_df['silhouette'], 'o-', linewidth=2, markersize=6)
    ax.set_xlabel('Weight Distance', fontsize=12)
    ax.set_ylabel('Silhouette Score', fontsize=12)
    ax.set_title('Silhouette Score vs Weight Distance', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

    # 2. Davies-Bouldin Index
    ax = axes[1]
    ax.plot(results_df['weight_distance'], results_df['davies_bouldin'], 'o-', linewidth=2, markersize=6, color='orange')
    ax.set_xlabel('Weight Distance', fontsize=12)
    ax.set_ylabel('Davies-Bouldin Index', fontsize=12)
    ax.set_title('Davies-Bouldin Index vs Weight Distance\n(чем ниже, тем лучше)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

    # 3. Boundary Ratio
    ax = axes[2]
    ax.plot(results_df['weight_distance'], results_df['boundary_ratio'], 'o-', linewidth=2, markersize=6, color='green')
    ax.set_xlabel('Weight Distance', fontsize=12)
    ax.set_ylabel('Boundary Ratio (%)', fontsize=12)
    ax.set_title('Boundary Ratio vs Weight Distance', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

    # 4. Boundary Density
    ax = axes[3]
    ax.plot(results_df['weight_distance'], results_df['boundary_density'], 'o-', linewidth=2, markersize=6, color='purple')
    ax.set_xlabel('Weight Distance', fontsize=12)
    ax.set_ylabel('Boundary Density', fontsize=12)
    ax.set_title('Boundary Density vs Weight Distance', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

    # 5. Boundary Alignment
    ax = axes[4]
    ax.plot(results_df['weight_distance'], results_df['boundary_alignment'], 'o-', linewidth=2, markersize=6, color='brown')
    ax.set_xlabel('Weight Distance', fontsize=12)
    ax.set_ylabel('Boundary Alignment', fontsize=12)
    ax.set_title('Boundary Alignment vs Weight Distance', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

    # 6. Heatmap: Silhouette vs Weights
    ax = axes[5]
    # Создаем сетку для heatmap
    weight_distances = results_df['weight_distance'].values
    silhouette_scores = results_df['silhouette'].values


    from scipy.interpolate import interp1d
    if len(weight_distances) > 1:
        f = interp1d(weight_distances, silhouette_scores, kind='linear', fill_value='extrapolate')
        weight_distances_smooth = np.linspace(weight_distances.min(), weight_distances.max(), 50)
        silhouette_smooth = f(weight_distances_smooth)

        ax.plot(weight_distances, silhouette_scores, 'o-', linewidth=2, markersize=8, label='Actual')
        ax.fill_between(weight_distances_smooth, silhouette_smooth, alpha=0.3, label='Interpolated')
        ax.set_xlabel('Weight Distance', fontsize=12)
        ax.set_ylabel('Silhouette Score', fontsize=12)
        ax.set_title('Silhouette Score: Detailed View', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()
    else:
        ax.text(0.5, 0.5, 'Недостаточно данных\nдля визуализации',
               ha='center', va='center', fontsize=12, transform=ax.transAxes)

    plt.suptitle('Анализ чувствительности к весам метрик', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

    return fig

print("Функции анализа чувствительности к весам определены")


Функции анализа чувствительности к весам определены


In [ ]:
# @title
def visualize_fuzzy_comparison(fuzzy_comparison_df, figsize=(18, 5)):
    """
    Визуализирует сравнение Fuzzy C-Means, Soft K-Means и VAE + KMeans + Boundary методов.

    Параметры:
    -----------
    fuzzy_comparison_df : pd.DataFrame
        DataFrame с метриками всех методов (из compare_with_fuzzy_methods)
    figsize : tuple
        Размер фигуры

    Возвращает:
    -----------
    fig : matplotlib.figure.Figure
        Объект фигуры
    """
    if fuzzy_comparison_df is None or fuzzy_comparison_df.empty:
        print("Внимание: данные для сравнения fuzzy методов недоступны")
        return None

    # Метрики для сравнения
    metrics_to_plot = ['silhouette', 'davies_bouldin', 'calinski_harabasz', 'boundary_ratio']

    # Проверяем, какие метрики доступны
    available_metrics = [m for m in metrics_to_plot if m in fuzzy_comparison_df.columns]

    if len(available_metrics) == 0:
        print("Внимание: нет доступных метрик для визуализации")
        return None


    n_metrics = len(available_metrics)
    fig, axes = plt.subplots(1, n_metrics, figsize=figsize)

    if n_metrics == 1:
        axes = [axes]

    methods = fuzzy_comparison_df['method'].values

    colors_map = {
        'Fuzzy C-Means': 'steelblue',
        'Soft K-Means': 'orange',
        'VAE + KMeans + Boundary': 'green'
    }

    # 1. Silhouette Score
    if 'silhouette' in available_metrics:
        idx = available_metrics.index('silhouette')
        ax = axes[idx]
        values = fuzzy_comparison_df['silhouette'].values
        colors = [colors_map.get(m, 'gray') for m in methods]
        bars = ax.barh(methods, values, color=colors, alpha=0.7)
        ax.set_xlabel('Silhouette Score', fontsize=12)
        ax.set_title('Silhouette Score\n(чем выше, тем лучше)', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        # Добавляем значения на столбцы
        for i, (bar, val) in enumerate(zip(bars, values)):
            ax.text(val + 0.01, i, f'{val:.3f}', va='center', fontsize=9)

    # 2. Davies-Bouldin Index
    if 'davies_bouldin' in available_metrics:
        idx = available_metrics.index('davies_bouldin')
        ax = axes[idx]
        values = fuzzy_comparison_df['davies_bouldin'].values
        colors = [colors_map.get(m, 'gray') for m in methods]
        bars = ax.barh(methods, values, color=colors, alpha=0.7)
        ax.set_xlabel('Davies-Bouldin Index', fontsize=12)
        ax.set_title('Davies-Bouldin Index\n(чем ниже, тем лучше)', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        # Добавляем значения на столбцы
        for i, (bar, val) in enumerate(zip(bars, values)):
            ax.text(val + 0.05, i, f'{val:.3f}', va='center', fontsize=9)

    # 3. Calinski-Harabasz Score
    if 'calinski_harabasz' in available_metrics:
        idx = available_metrics.index('calinski_harabasz')
        ax = axes[idx]
        values = fuzzy_comparison_df['calinski_harabasz'].values
        colors = [colors_map.get(m, 'gray') for m in methods]
        bars = ax.barh(methods, values, color=colors, alpha=0.7)
        ax.set_xlabel('Calinski-Harabasz Score', fontsize=12)
        ax.set_title('Calinski-Harabasz Score\n(чем выше, тем лучше)', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')

        for i, (bar, val) in enumerate(zip(bars, values)):
            if val > 1000:
                label = f'{val/1000:.1f}K'
            else:
                label = f'{val:.0f}'
            ax.text(val + val*0.02, i, label, va='center', fontsize=9)

    # 4. Boundary Ratio
    if 'boundary_ratio' in available_metrics:
        idx = available_metrics.index('boundary_ratio')
        ax = axes[idx]
        values = fuzzy_comparison_df['boundary_ratio'].values
        colors = [colors_map.get(m, 'gray') for m in methods]
        bars = ax.barh(methods, values, color=colors, alpha=0.7)
        ax.set_xlabel('Boundary Ratio (%)', fontsize=12)
        ax.set_title('Boundary Ratio\n(целевой диапазон 5-15%)', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        # Добавляем значения на столбцы
        for i, (bar, val) in enumerate(zip(bars, values)):
            ax.text(val + 0.5, i, f'{val:.2f}%', va='center', fontsize=9)

    plt.suptitle('Сравнение Fuzzy методов с VAE + KMeans + Boundary',
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    return fig

print("Функция визуализации сравнения fuzzy методов определена")


Функция визуализации сравнения fuzzy методов определена


## Сравнение с методами мягкой кластеризации

Мягкая кластеризация также позволяет работать с неопределенностью принадлежности точек к кластерам, что концептуально близко к нашей идее boundary кластеров.

### Fuzzy C-Means

**Принцип**: каждая точка имеет степень принадлежности к каждому кластеру, а не жесткую метку.

**Boundary точки в Fuzzy C-Means**:
- Определяются через энтропию принадлежностей
- Высокая энтропия = неопределенность = boundary точка
- Группируются с помощью DBSCAN

### Soft K-Means

**Принцип**: итеративное обновление центров кластеров с учетом мягких принадлежностей точек.

**Отличия от нашего метода**:
- Fuzzy методы работают на исходном пространстве признаков
- Наш метод использует латентное пространство VAE для лучшей разделимости
- Наш метод явно группирует boundary точки в отдельные кластеры

### Методологическое сравнение

Сравнение показывает:
- Преимущества использования латентного пространства VAE
- Эффективность интегральной boundary метрики
- Практическую применимость boundary кластеров для бизнес-задач


In [ ]:
# @title
try:
    from sklearn_extensions.fuzzy_kmeans import FuzzyKMeans
    FUZZY_KMEANS_AVAILABLE = True
except ImportError:
    try:
        import skfuzzy as fuzz
        FUZZY_C_MEANS_AVAILABLE = True
        FUZZY_KMEANS_AVAILABLE = False
    except ImportError:
        FUZZY_C_MEANS_AVAILABLE = False
        FUZZY_KMEANS_AVAILABLE = False
        print("Библиотеки для fuzzy clustering недоступны.")
        print("   Для установки: pip install scikit-fuzzy")
        print("   Или: pip install sklearn-extensions")

try:
    from sklearn.cluster import SpectralClustering
    SPECTRAL_AVAILABLE = True
except ImportError:
    SPECTRAL_AVAILABLE = False

def compare_with_fuzzy_methods(X, results, n_clusters=None, random_state=None):
    """
    Сравнивает наш метод с методами мягкой кластеризации (fuzzy C-means, soft K-means).

    Параметры:
    -----------
    X : np.ndarray
        Предобработанные данные
    results : dict
        Результаты VAE + KMeans + Boundary метода
    n_clusters : int, optional
        Количество кластеров. Если None, берется из results
    random_state : int, optional
        Random state для воспроизводимости

    Возвращает:
    -----------
    fuzzy_results : dict
        Словарь с результатами fuzzy методов
    comparison_df : pd.DataFrame
        DataFrame с метриками всех методов
    """
    if random_state is None:
        random_state = RANDOM_STATE

    if n_clusters is None:
        n_clusters = len(np.unique(results['cluster_labels']))

    fuzzy_results = {}
    comparison_data = []

    print("="*80)
    print("СРАВНЕНИЕ С FUZZY МЕТОДАМИ")
    print("="*80)

    # 1. Fuzzy C-Means
    if FUZZY_C_MEANS_AVAILABLE:
        try:
            print("\n1. Fuzzy C-Means (scikit-fuzzy)...")
            start_time = time.time()

            # Транспонируем данные для scikit-fuzzy
            X_fuzzy = X.T

            # Инициализация центров кластеров
            cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
                X_fuzzy, n_clusters, 2, error=0.005, maxiter=1000, init=None, seed=random_state
            )

            # Получаем метки кластеров (кластер с максимальной принадлежностью)
            fuzzy_labels = np.argmax(u, axis=0)

            membership_entropy = entropy(u, axis=0)
            # Нормализуем энтропию
            membership_entropy_norm = (membership_entropy - membership_entropy.min()) / (membership_entropy.max() - membership_entropy.min() + 1e-10)

            # Boundary точки: высокое значение энтропии принадлежностей
            boundary_threshold_fuzzy = 0.5
            fuzzy_boundary_mask = membership_entropy_norm >= boundary_threshold_fuzzy

            # Создаем boundary labels (используем DBSCAN для группировки boundary точек)
            fuzzy_boundary_labels = np.full(len(fuzzy_labels), -1)
            if fuzzy_boundary_mask.sum() > 0:
                boundary_vectors_fuzzy = X[fuzzy_boundary_mask]
                if len(boundary_vectors_fuzzy) > 1:
                    # Используем DBSCAN для группировки boundary точек
                    nbrs_fuzzy = NearestNeighbors(n_neighbors=min(10, len(boundary_vectors_fuzzy))).fit(boundary_vectors_fuzzy)
                    distances_fuzzy, _ = nbrs_fuzzy.kneighbors(boundary_vectors_fuzzy)
                    k_distances_fuzzy = np.sort(distances_fuzzy[:, -1])
                    eps_fuzzy = np.percentile(k_distances_fuzzy, 90)
                    min_samples_fuzzy = max(5, len(boundary_vectors_fuzzy) // 20)

                    dbscan_fuzzy = DBSCAN(eps=eps_fuzzy, min_samples=min_samples_fuzzy)
                    boundary_cluster_labels_fuzzy = dbscan_fuzzy.fit_predict(boundary_vectors_fuzzy)

                    boundary_idx = 0
                    for i in range(len(fuzzy_labels)):
                        if fuzzy_boundary_mask[i]:
                            if boundary_cluster_labels_fuzzy[boundary_idx] != -1:
                                fuzzy_boundary_labels[i] = boundary_cluster_labels_fuzzy[boundary_idx] + n_clusters
                            boundary_idx += 1

            fuzzy_time = time.time() - start_time

            # Вычисляем метрики
            # Метрики вычисляются на исходном пространстве (X)
            fuzzy_metrics = evaluate_clustering_metrics(X, fuzzy_labels, fuzzy_boundary_labels)
            fuzzy_metrics['method'] = 'Fuzzy C-Means'
            fuzzy_metrics['execution_time'] = fuzzy_time
            fuzzy_metrics['fpc'] = fpc  # Fuzzy partition coefficient

            fuzzy_results['Fuzzy C-Means'] = {
                'labels': fuzzy_labels,
                'boundary_labels': fuzzy_boundary_labels,
                'membership_matrix': u,
                'metrics': fuzzy_metrics
            }

            comparison_data.append(fuzzy_metrics)
            print(f"   Время выполнения: {fuzzy_time:.2f} сек")
            print(f"   Fuzzy Partition Coefficient (FPC): {fpc:.4f} (чем выше, тем лучше)")
            print(f"   Silhouette Score: {fuzzy_metrics['silhouette']:.4f}")
            print(f"   Boundary точек: {fuzzy_boundary_mask.sum()} ({fuzzy_boundary_mask.sum()/len(fuzzy_labels)*100:.2f}%)")

        except Exception as e:
            print(f"   Ошибка при выполнении Fuzzy C-Means: {e}")

    # 2. Soft K-Means (используя sklearn-extensions или собственную реализацию)
    print("\n2. Soft K-Means (собственная реализация)...")
    try:
        start_time = time.time()

        # Инициализация центров (используем KMeans)
        kmeans_init = KMeans(n_clusters=n_clusters, n_init=1, random_state=random_state)
        kmeans_init.fit(X)
        centers = kmeans_init.cluster_centers_

        # Soft K-Means: итеративное обновление центров с мягкими принадлежностями
        # Используем экспоненциальное взвешивание для мягких меток
        beta = 1.0  # Параметр температуры (чем больше, тем "жестче" кластеризация)
        max_iter = 100
        tolerance = 1e-4

        for iteration in range(max_iter):
            # Вычисляем расстояния до всех центров
            distances = cdist(X, centers)

            # Мягкие принадлежности: exp(-beta * distance) / sum(exp(-beta * distance))
            exp_distances = np.exp(-beta * distances)
            soft_memberships = exp_distances / exp_distances.sum(axis=1, keepdims=True)

            # Обновляем центры как взвешенное среднее
            new_centers = (soft_memberships.T @ X) / soft_memberships.sum(axis=0, keepdims=True).T

            # Проверяем сходимость
            if np.linalg.norm(new_centers - centers) < tolerance:
                break
            centers = new_centers

        # Получаем жесткие метки (кластер с максимальной принадлежностью)
        soft_labels = np.argmax(soft_memberships, axis=1)

        # Boundary точки: точки с высокой энтропией принадлежностей
        membership_entropy_soft = entropy(soft_memberships, axis=1)
        membership_entropy_soft_norm = (membership_entropy_soft - membership_entropy_soft.min()) / (membership_entropy_soft.max() - membership_entropy_soft.min() + 1e-10)

        boundary_threshold_soft = 0.5
        soft_boundary_mask = membership_entropy_soft_norm >= boundary_threshold_soft

        # Создаем boundary labels
        soft_boundary_labels = np.full(len(soft_labels), -1)
        if soft_boundary_mask.sum() > 0:
            boundary_vectors_soft = X[soft_boundary_mask]
            if len(boundary_vectors_soft) > 1:
                nbrs_soft = NearestNeighbors(n_neighbors=min(10, len(boundary_vectors_soft))).fit(boundary_vectors_soft)
                distances_soft, _ = nbrs_soft.kneighbors(boundary_vectors_soft)
                k_distances_soft = np.sort(distances_soft[:, -1])
                eps_soft = np.percentile(k_distances_soft, 90)
                min_samples_soft = max(5, len(boundary_vectors_soft) // 20)

                dbscan_soft = DBSCAN(eps=eps_soft, min_samples=min_samples_soft)
                boundary_cluster_labels_soft = dbscan_soft.fit_predict(boundary_vectors_soft)

                boundary_idx = 0
                for i in range(len(soft_labels)):
                    if soft_boundary_mask[i]:
                        if boundary_cluster_labels_soft[boundary_idx] != -1:
                            soft_boundary_labels[i] = boundary_cluster_labels_soft[boundary_idx] + n_clusters
                        boundary_idx += 1

        soft_time = time.time() - start_time

        # Вычисляем метрики
        # Метрики вычисляются на исходном пространстве (X)
        soft_metrics = evaluate_clustering_metrics(X, soft_labels, soft_boundary_labels)
        soft_metrics['method'] = 'Soft K-Means'
        soft_metrics['execution_time'] = soft_time
        soft_metrics['avg_entropy'] = membership_entropy_soft.mean()  # Средняя энтропия принадлежностей

        fuzzy_results['Soft K-Means'] = {
            'labels': soft_labels,
            'boundary_labels': soft_boundary_labels,
            'membership_matrix': soft_memberships,
            'metrics': soft_metrics
        }

        comparison_data.append(soft_metrics)
        print(f"   Время выполнения: {soft_time:.2f} сек")
        print(f"   Средняя энтропия принадлежностей: {soft_metrics['avg_entropy']:.4f}")
        print(f"   Silhouette Score: {soft_metrics['silhouette']:.4f}")
        print(f"   Boundary точек: {soft_boundary_mask.sum()} ({soft_boundary_mask.sum()/len(soft_labels)*100:.2f}%)")

    except Exception as e:
        print(f"   Ошибка при выполнении Soft K-Means: {e}")

    # 3. Наш метод (VAE + KMeans + Boundary)
    vae_metrics = results['metrics'].copy()
    vae_metrics['method'] = 'VAE + KMeans + Boundary'
    if 'execution_time' in results:
        vae_metrics['execution_time'] = results['execution_time']
    comparison_data.append(vae_metrics)

    comparison_df = pd.DataFrame(comparison_data)

    print("\n" + "="*80)
    print("ИТОГОВОЕ СРАВНЕНИЕ")
    print("="*80)
    print(comparison_df[['method', 'silhouette', 'davies_bouldin', 'calinski_harabasz',
                         'boundary_ratio', 'execution_time']].to_string(index=False))

    return fuzzy_results, comparison_df

print("Функция сравнения с fuzzy методами определена")


Функция сравнения с fuzzy методами определена


## Запуск


In [ ]:
# @title
df = pd.read_csv('online_shoppers_intention.csv')

results = run_vae_boundary_clustering(
    df,
    exclude_columns=None,  # Столбцы для исключения (например, ID)
    latent_dim=20,  # Размерность латентного пространства
    hidden_dims=[512, 256, 128, 64],  # Архитектура скрытых слоев VAE
    n_clusters=4,  # Количество основных кластеров (оптимизируется через optimize=True)
    boundary_threshold=0.5,  # Порог для определения boundary точек (0-1)
    max_boundary_clusters=None,  # Максимальное количество boundary кластеров (None = ограничено n_clusters)
    vae_epochs=250,  # Количество эпох обучения VAE
    vae_beta=2.5,  # Коэффициент β для β-VAE (было 4.0)
    vae_batch_size=256,  # Размер батча для обучения VAE
    vae_lr=5e-4,  # Learning rate для обучения VAE
    vae_dropout=0.2,  # Вероятность dropout для VAE
    use_batch_norm=True,  # Использовать ли Batch Normalization
    early_stopping_patience=30,  # Количество эпох без улучшения
    optimize=True,  # Включить Bayesian Optimization
    max_optimization_iterations=15,  # Количество итераций оптимизации
    n_clusters_range=(3, 6),  # Диапазон количества кластеров для оптимизации (min, max)
    # Если None, используется диапазон вокруг n_clusters: (max(2, n_clusters-2), min(n_clusters+3, 15))
    verbose=True  # Выводить подробную информацию
)

# ========== 1. ВИЗУАЛИЗАЦИЯ НАШЕЙ МОДЕЛИ ==========
print("\n" + "="*80)
print("ВИЗУАЛИЗАЦИЯ НАШЕЙ МОДЕЛИ")
print("="*80)

# График истории обучения VAE
print("\nГрафик истории обучения VAE:")
plot_training_history(results)

# 2D и 3D визуализация нашей модели
visualize_clusters(results, method_name="VAE + KMeans + Boundary")

# ========== 1.5. АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ К ВЕСАМ МЕТРИК ==========

weight_sensitivity_results = analyze_weight_sensitivity(
    results['latent_vectors'],
    results['cluster_labels'],
    results['X_processed'],
    boundary_threshold=0.5
)

# Визуализация результатов анализа чувствительности
visualize_weight_sensitivity(weight_sensitivity_results)

# ========== 1.6. СРАВНЕНИЕ С FUZZY МЕТОДАМИ ==========

# Сравнение с методами мягкой кластеризации
fuzzy_results, fuzzy_comparison_df = compare_with_fuzzy_methods(
    results['X_processed'],
    results,
    n_clusters=len(np.unique(results['cluster_labels']))
)

# Визуализация сравнения fuzzy методов
if fuzzy_comparison_df is not None and not fuzzy_comparison_df.empty:
    print("\nВизуализация сравнения Fuzzy методов:")
    visualize_fuzzy_comparison(fuzzy_comparison_df)

# ========== 2. СРАВНЕНИЕ С BASELINE МЕТОДАМИ ==========
# Сравнение методов
comparison_df, all_metrics_runs = compare_with_baselines(
    results['X_processed'],
    results,
    n_runs=10
)

# Визуализация сравнения методов в 2D
visualize_methods_comparison_2d_no_boundary(results['X_processed'], results)

# Визуализация сравнения методов в 3D
visualize_methods_comparison_3d(results['X_processed'], results)

# Визуализация сравнения метрик
fig_metrics, _ = visualize_metrics_comparison(results['X_processed'], results, comparison_df)

# ========== 3. СТАТИСТИЧЕСКОЕ СРАВНЕНИЕ ==========

# Статистическое сравнение методов
pvalue_matrices, friedman_results = statistical_comparison(
    all_metrics_runs,
    metrics_to_compare=['silhouette', 'davies_bouldin', 'calinski_harabasz'],
    correction_method='bonferroni'  # Можно использовать 'fdr_bh' для менее консервативной коррекции
)

# Тепловые карты p-values
plot_pvalue_heatmap(
    pvalue_matrices,
    significance_level=0.05
)

# ========== 4. ОПИСАНИЕ КЛАСТЕРОВ ==========

# Вывод описания кластеров
print_cluster_descriptions(
    results['cluster_descriptions'],
    results['cluster_statistics'],
    verbose=True
)

# ========== 5. ИТОГОВЫЕ РЕЗУЛЬТАТЫ ==========
print("\n" + "="*80)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ ИССЛЕДОВАНИЯ")
print("="*80)

print(f"\nОсновные результаты:")
print(f"   - Найдено основных кластеров: {len(np.unique(results['cluster_labels']))}")
print(f"   - Найдено boundary кластеров: {len(np.unique(results['boundary_labels'][results['boundary_labels'] != -1]))}")
print(f"   - Boundary точек: {(results['boundary_labels'] != -1).sum()} ({(results['boundary_labels'] != -1).sum()/len(df)*100:.2f}%)")

print(f"\nМетрики кластеризации:")
for metric, value in results['metrics'].items():
    if not np.isnan(value) and not np.isinf(value):
        print(f"   - {metric:25s}: {value:.4f}")

print(f"\nСравнение с базовыми методами:")
print(comparison_df[['method', 'silhouette', 'davies_bouldin', 'calinski_harabasz']].to_string(index=False))

# Добавляем информацию о fuzzy методах
if 'fuzzy_comparison_df' in locals() and fuzzy_comparison_df is not None:
    print(f"\nСравнение с fuzzy методами:")
    print(fuzzy_comparison_df[['method', 'silhouette', 'davies_bouldin', 'calinski_harabasz', 'boundary_ratio']].to_string(index=False))

# Добавляем информацию о лучших весах из анализа чувствительности
if 'weight_sensitivity_results' in locals() and weight_sensitivity_results is not None:
    best_weight_idx = weight_sensitivity_results['silhouette'].idxmax()
    best_weight_row = weight_sensitivity_results.loc[best_weight_idx]
    print(f"\nАнализ чувствительности к весам:")
    print(f"   Лучшая комбинация весов:")
    print(f"   - weight_distance: {best_weight_row['weight_distance']:.2f}")
    print(f"   - weight_entropy: {best_weight_row['weight_entropy']:.2f}")
    print(f"   - Silhouette Score: {best_weight_row['silhouette']:.4f}")
    print(f"   - Boundary Ratio: {best_weight_row['boundary_ratio']:.2f}%")


Output hidden; open in https://colab.research.google.com to view.